In [287]:
#EDA
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#to download json_lines: python -m pip install json-lines
#pip install textstat
#pip install --upgrade gensim
from pathlib import Path
import json_lines, json, copy
import numpy as np
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import gensim.downloader as api
import gensim
import operator
from nltk.util import ngrams 
import nltk
from nltk.corpus import stopwords
from nltk import download
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from gensim import corpora

import csv
from textstat.textstat import textstat
stop_words = stopwords.words('english')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/EthanHsu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [288]:
raw_instance_filePath = Path("../clickbait_dataset/instances_train.jsonl")
raw_instance_dir = Path("../clickbait_dataset")
raw_truth_filePath = Path("../clickbait_dataset/truth_train.jsonl")
raw_truth_dir = Path("../clickbait_dataset")

In [289]:
#open with json - raw_instance_train
raw_instance_train=[]
with open('../clickbait_dataset/instances_train.jsonl','rb') as raw_instance_file: #opening file with binary(rb) mode.
    for x in raw_instance_file:
        raw_instance_train.append(json.loads(x))
raw_instance_train[0]

### Fields in instances.jsonl:
##{
#"id" : "<instance id>",
#"postTimestamp" : "<weekday> <month> <day> <hour>:<minute>:<second><time_offset> <year>",
#"postText" : ["<text of the tweet post with links removed>"],
#"postMedia" : ["<path to a file in the media archive>"],
#"targetTitle" : "<title of target news article>",
#"targetDescription": "<description tag of target news article>",
#"targetKeywords" : "<keywords tag of target news article>",
#"targetParagraphs" : ["<text of the i-th paragraph in the target article>"],
#"targetCaptions" : ["<caption of the i-th image in the target article>"]}
### Fields in truth.jsonl:
#{
#"id" : "<instance id>",
#"truthJudgments" : [<number in [0,1]>],
#"truthMean" : <number in [0,1]>,
#"truthMedian" : <number in [0,1]>,
#"truthMode" : <number in [0,1]>,
#truthClass" : "clickbait | no-clickbait"
#}

{'id': '0',
 'postMedia': [],
 'postText': ["Apple's iOS 9 'App thinning' feature will give your phone's storage a boost"],
 'postTimestamp': 'Tue Jun 09 16:31:10 +0000 2015',
 'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The Ap

In [290]:
for n in raw_instance_train:
    print(n['id'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3223
3224
3225
3226
3227
3228
3229
3230
3231
3232
3233
3234
3235
3236
3237
3238
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249
3250
3251
3252
3253
3254
3255
3256
3257
3258
3259
3260
3261
3262
3263
3264
3265
3266
3267
3268
3269
3270
3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283
3284
3285
3286
3287
3288
3289
3290
3291
3292
3293
3294
3295
3296
3297
3298
3299
3300
3301
3302
3303
3304
3305
3306
3307
3308
3309
3310
3311
3312
3313
3314
3315
3316
3317
3318
3319
3320
3321
3322
3323
3324
3325
3326
3327
3328
3329
3330
3331
3332
3333
3334
3335
3336
3337
3338
3339
3340
3341
3342
3343
3344
3345
3346
3347
3348
3349
3350
3351
3352
3353
3354
3355
3356
3357
3358
3359
3360
3361
3362
3363
3364
3365
3366
3367
3368
3369
3370
3371
3372
3373
3374
3375
3376
3377
3378
3379
3380
3381
3382
3383


4879
4880
4881
4882
4883
4884
4885
4886
4887
4888
4889
4890
4891
4892
4893
4894
4895
4896
4897
4898
4899
4900
4901
4902
4903
4904
4905
4906
4907
4908
4909
4910
4911
4912
4913
4914
4915
4916
4917
4918
4919
4920
4921
4922
4923
4924
4925
4926
4927
4928
4929
4930
4931
4932
4933
4934
4935
4936
4937
4938
4939
4940
4941
4942
4943
4944
4945
4946
4947
4948
4949
4950
4951
4952
4953
4954
4955
4956
4957
4958
4959
4960
4961
4962
4963
4964
4965
4966
4967
4968
4969
4970
4971
4972
4973
4974
4975
4976
4977
4978
4979
4980
4981
4982
4983
4984
4985
4986
4987
4988
4989
4990
4991
4992
4993
4994
4995
4996
4997
4998
4999
5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078


7773
7774
7775
7776
7777
7778
7779
7780
7781
7782
7783
7784
7785
7786
7787
7788
7789
7790
7791
7792
7793
7794
7795
7796
7797
7798
7799
7800
7801
7802
7803
7804
7805
7806
7807
7808
7809
7810
7811
7812
7813
7814
7815
7816
7817
7818
7819
7820
7821
7822
7823
7824
7825
7826
7827
7828
7829
7830
7831
7832
7833
7834
7835
7836
7837
7838
7839
7840
7841
7842
7843
7844
7845
7846
7847
7848
7849
7850
7851
7852
7853
7854
7855
7856
7857
7858
7859
7860
7861
7862
7863
7864
7865
7866
7867
7868
7869
7870
7871
7872
7873
7874
7875
7876
7877
7878
7879
7880
7881
7882
7883
7884
7885
7886
7887
7888
7889
7890
7891
7892
7893
7894
7895
7896
7897
7898
7899
7900
7901
7902
7903
7904
7905
7906
7907
7908
7909
7910
7911
7912
7913
7914
7915
7916
7917
7918
7919
7920
7921
7922
7923
7924
7925
7926
7927
7928
7929
7930
7931
7932
7933
7934
7935
7936
7937
7938
7939
7940
7941
7942
7943
7944
7945
7946
7947
7948
7949
7950
7951
7952
7953
7954
7955
7956
7957
7958
7959
7960
7961
7962
7963
7964
7965
7966
7967
7968
7969
7970
7971
7972


10708
10709
10710
10711
10712
10713
10714
10715
10716
10717
10718
10719
10720
10721
10722
10723
10724
10725
10726
10727
10728
10729
10730
10731
10732
10733
10734
10735
10736
10737
10738
10739
10740
10741
10742
10743
10744
10745
10746
10747
10748
10749
10750
10751
10752
10753
10754
10755
10756
10757
10758
10759
10760
10761
10762
10763
10764
10765
10766
10767
10768
10769
10770
10771
10772
10773
10774
10775
10776
10777
10778
10779
10780
10781
10782
10783
10784
10785
10786
10787
10788
10789
10790
10791
10792
10793
10794
10795
10796
10797
10798
10799
10800
10801
10802
10803
10804
10805
10806
10807
10808
10809
10810
10811
10812
10813
10814
10815
10816
10817
10818
10819
10820
10821
10822
10823
10824
10825
10826
10827
10828
10829
10830
10831
10832
10833
10834
10835
10836
10837
10838
10839
10840
10841
10842
10843
10844
10845
10846
10847
10848
10849
10850
10851
10852
10853
10854
10855
10856
10857
10858
10859
10860
10861
10862
10863
10864
10865
10866
10867
10868
10869
10870
10871
10872
10873
1087

12263
12264
12265
12266
12267
12268
12269
12270
12271
12272
12273
12274
12275
12276
12277
12278
12279
12280
12281
12282
12283
12284
12285
12286
12287
12288
12289
12290
12291
12292
12293
12294
12295
12296
12297
12298
12299
12300
12301
12302
12303
12304
12305
12306
12307
12308
12309
12310
12311
12312
12313
12314
12315
12316
12317
12318
12319
12320
12321
12322
12323
12324
12325
12326
12327
12328
12329
12330
12331
12332
12333
12334
12335
12336
12337
12338
12339
12340
12341
12342
12343
12344
12345
12346
12347
12348
12349
12350
12351
12352
12353
12354
12355
12356
12357
12358
12359
12360
12361
12362
12363
12364
12365
12366
12367
12368
12369
12370
12371
12372
12373
12374
12375
12376
12377
12378
12379
12380
12381
12382
12383
12384
12385
12386
12387
12388
12389
12390
12391
12392
12393
12394
12395
12396
12397
12398
12399
12400
12401
12402
12403
12404
12405
12406
12407
12408
12409
12410
12411
12412
12413
12414
12415
12416
12417
12418
12419
12420
12421
12422
12423
12424
12425
12426
12427
12428
1242

13864
13865
13866
13867
13868
13869
13870
13871
13872
13873
13874
13875
13876
13877
13878
13879
13880
13881
13882
13883
13884
13885
13886
13887
13888
13889
13890
13891
13892
13893
13894
13895
13896
13897
13898
13899
13900
13901
13902
13903
13904
13905
13906
13907
13908
13909
13910
13911
13912
13913
13914
13915
13916
13917
13918
13919
13920
13921
13922
13923
13924
13925
13926
13927
13928
13929
13930
13931
13932
13933
13934
13935
13936
13937
13938
13939
13940
13941
13942
13943
13944
13945
13946
13947
13948
13949
13950
13951
13952
13953
13954
13955
13956
13957
13958
13959
13960
13961
13962
13963
13964
13965
13966
13967
13968
13969
13970
13971
13972
13973
13974
13975
13976
13977
13978
13979
13980
13981
13982
13983
13984
13985
13986
13987
13988
13989
13990
13991
13992
13993
13994
13995
13996
13997
13998
13999
14000
14001
14002
14003
14004
14005
14006
14007
14008
14009
14010
14011
14012
14013
14014
14015
14016
14017
14018
14019
14020
14021
14022
14023
14024
14025
14026
14027
14028
14029
1403

15366
15367
15368
15369
15370
15371
15372
15373
15374
15375
15376
15377
15378
15379
15380
15381
15382
15383
15384
15385
15386
15387
15388
15389
15390
15391
15392
15393
15394
15395
15396
15397
15398
15399
15400
15401
15402
15403
15404
15405
15406
15407
15408
15409
15410
15411
15412
15413
15414
15415
15416
15417
15418
15419
15420
15421
15422
15423
15424
15425
15426
15427
15428
15429
15430
15431
15432
15433
15434
15435
15436
15437
15438
15439
15440
15441
15442
15443
15444
15445
15446
15447
15448
15449
15450
15451
15452
15453
15454
15455
15456
15457
15458
15459
15460
15461
15462
15463
15464
15465
15466
15467
15468
15469
15470
15471
15472
15473
15474
15475
15476
15477
15478
15479
15480
15481
15482
15483
15484
15485
15486
15487
15488
15489
15490
15491
15492
15493
15494
15495
15496
15497
15498
15499
15500
15501
15502
15503
15504
15505
15506
15507
15508
15509
15510
15511
15512
15513
15514
15515
15516
15517
15518
15519
15520
15521
15522
15523
15524
15525
15526
15527
15528
15529
15530
15531
1553

16743
16744
16745
16746
16747
16748
16749
16750
16751
16752
16753
16754
16755
16756
16757
16758
16759
16760
16761
16762
16763
16764
16765
16766
16767
16768
16769
16770
16771
16772
16773
16774
16775
16776
16777
16778
16779
16780
16781
16782
16783
16784
16785
16786
16787
16788
16789
16790
16791
16792
16793
16794
16795
16796
16797
16798
16799
16800
16801
16802
16803
16804
16805
16806
16807
16808
16809
16810
16811
16812
16813
16814
16815
16816
16817
16818
16819
16820
16821
16822
16823
16824
16825
16826
16827
16828
16829
16830
16831
16832
16833
16834
16835
16836
16837
16838
16839
16840
16841
16842
16843
16844
16845
16846
16847
16848
16849
16850
16851
16852
16853
16854
16855
16856
16857
16858
16859
16860
16861
16862
16863
16864
16865
16866
16867
16868
16869
16870
16871
16872
16873
16874
16875
16876
16877
16878
16879
16880
16881
16882
16883
16884
16885
16886
16887
16888
16889
16890
16891
16892
16893
16894
16895
16896
16897
16898
16899
16900
16901
16902
16903
16904
16905
16906
16907
16908
1690

In [291]:
raw_truth_train=[]
with open('../clickbait_dataset/truth_train.jsonl','rb') as raw_truth_file: #opening file with binary(rb) mode.
    for x in raw_truth_file:
        raw_truth_train.append(json.loads(x))

In [276]:
for n in raw_truth_train:
    print(n['id'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

14596
16159
2085
5228
6760
3681
3682
8323
6761
14597
8324
14598
11450
8325
13052
5229
3683
8326
8327
8328
6762
13053
8329
3684
6763
13054
3685
14599
3686
13055
14600
6764
9863
13056
2086
5230
14601
6765
16160
13057
6766
11451
8330
6767
16161
14602
6768
2087
8331
5231
13058
8332
2088
9864
3687
6769
9865
16162
5232
14603
2089
13059
11452
3688
9866
11453
9867
16163
16164
16165
13060
14604
16166
8333
16167
5233
16168
14605
2090
14606
9868
8334
14607
3689
14608
11454
13061
5234
6770
13062
2091
13063
9869
2092
5235
3690
2093
3691
2094
9870
16169
14609
14610
11455
13064
11456
14611
13065
14612
14613
11457
5236
11458
3692
8335
8336
8337
11459
2095
16170
16171
11460
6771
8338
8339
16172
16173
6772
6773
6774
16174
13066
13067
11461
11462
14614
2096
3693
9871
13068
16175
2097
11463
11464
8340
2098
5237
13069
8341
6775
14615
11465
14616
14617
9872
2099
16176
2100
16177
5238
13070
13071
6776
8342
2101
13072
3694
16178
2102
16179
14618
11466
14619
9873
14620
16180
5239
9874
11467
2103
8343
5240
8344

10003
14768
2264
2265
14769
3847
5380
10004
16298
8473
11632
8474
10005
11633
3848
10006
6912
16299
2266
11634
6913
16300
16301
2267
2268
14770
16302
3849
5381
10007
3850
6914
16303
8475
16304
5382
10008
6915
8476
16305
11635
10009
10010
13218
2269
16306
10011
10012
11636
8477
8478
13219
13220
16307
10013
13221
3851
2270
16308
3852
10014
10015
8479
3853
13222
3854
14771
3855
2271
13223
11637
3856
8480
16309
13224
13225
6916
13226
16310
11638
13227
10016
16311
13228
11639
6917
5383
6918
2272
8481
6919
5384
13229
8482
14772
16312
13230
8483
13231
16313
13232
10017
16314
2273
8484
5385
8485
2274
10018
6920
3857
10019
10020
5386
2275
10021
5387
11640
2276
14773
13233
11641
14774
6921
13234
11642
6922
3858
3859
14775
3860
10022
6923
13235
6924
5388
13236
8486
2277
11643
16315
8487
6925
3861
14776
16316
10023
16317
6926
3862
16318
2278
6927
11644
13237
3863
2279
16319
16320
3864
14777
10024
13238
11645
6928
10025
13239
8488
10026
11646
2280
10027
14778
3865
5389
16321
3866
13240
8489
16322
8

5659
13512
16567
5660
2552
7196
2553
10318
4121
15072
8767
15073
15074
8768
11914
8769
16568
15075
7197
7198
13513
10319
7199
5661
7200
15076
13514
8770
2554
5662
16569
2555
8771
13515
16570
8772
16571
13516
2556
10320
5663
16572
11915
8773
7201
16573
13517
8774
15077
10321
4122
10322
10323
7202
5664
8775
11916
16574
4123
10324
4124
11917
16575
13518
7203
4125
5665
8776
7204
16576
8777
8778
10325
4126
5666
10326
8779
10327
11918
8780
5667
13519
16577
15078
7205
15079
10328
15080
10329
7206
13520
2557
15081
13521
11919
16578
13522
13523
11920
15082
16579
16580
7207
8781
11921
4127
10330
15083
10331
13524
13525
5668
16581
5669
2558
8782
4128
5670
8783
11922
11923
7208
13526
10332
15084
15085
16582
7209
2559
13527
8784
4129
5671
10333
10334
15086
4130
13528
15087
5672
4131
16583
13529
15088
16584
5673
10335
8785
8786
10336
7210
16585
15089
10337
15090
13530
16586
15091
4132
11924
2560
8787
8788
13531
13532
15092
5674
11925
10338
10339
7211
2561
10340
15093
2562
11926
10341
16587
15094
119

5952
10584
16858
10585
16859
5953
2808
12177
2809
5954
13759
9052
12178
16860
13760
10586
16861
16862
16863
10587
15318
15319
4398
10588
9053
5955
13761
7473
7474
5956
12179
9054
16864
15320
7475
13762
12180
7476
12181
16865
12182
15321
2810
2811
16866
10589
2812
7477
7478
4399
7479
5957
13763
2813
13764
12183
5958
2814
5959
9055
5960
7480
15322
9056
9057
15323
7481
9058
2815
12184
16867
2816
9059
5961
9060
2817
12185
4400
2818
5962
9061
10590
10591
10592
2819
15324
5963
9062
9063
7482
13765
4401
9064
15325
13766
12186
13767
16868
2820
12187
2821
2822
2823
10593
9065
16869
10594
7483
7484
9066
9067
7485
16870
10595
16871
12188
16872
5964
12189
10596
7486
5965
13768
4402
10597
2824
16873
16874
4403
4404
5966
7487
10598
10599
16875
4405
10600
15326
15327
15328
2825
5967
9068
7488
7489
10601
15329
9069
10602
10603
13769
2826
15330
13770
5968
10604
4406
9070
4407
16876
7490
5969
16877
12190
2827
4408
12191
12192
5970
2828
9071
5971
10605
5972
12193
12194
16878
4409
12195
4410
16879
9072
10

6171
10814
3045
9289
6172
7703
6173
7704
3046
4616
4617
12410
6174
17082
17083
17084
10815
17085
12411
4618
7705
7706
15531
10816
17086
15532
3047
13965
10817
12412
9290
17087
17088
12413
12414
17089
13966
4619
7707
10818
15533
13967
13968
6175
17090
4620
10819
10820
3048
4621
10821
17091
6176
6177
7708
7709
3049
15534
6178
9291
12415
7710
6179
6180
12416
12417
15535
9292
12418
12419
4622
10822
17092
10823
17093
7711
10824
10825
15536
12420
7712
9293
17094
10826
4623
7713
7714
15537
10827
10828
10829
3050
6181
4624
17095
15538
3051
13969
3052
12421
3053
13970
12422
3054
10830
17096
9294
9295
10831
13971
10832
6182
9296
9297
3055
17097
15539
7715
6183
4625
6184
4626
4627
10833
7716
3056
6185
10834
3057
17098
10835
13972
6186
15540
3058
7717
15541
4628
6187
4629
3059
17099
4630
7718
3060
17100
15542
12423
12424
7719
10836
4631
17101
10837
7720
9298
6188
10838
7721
12425
15543
7722
17102
15544
17103
9299
13973
6189
9300
4632
17104
7723
7724
9301
12426
9302
17105
3061
6190
4633
13974
4634


17333
11078
7964
15776
9538
6409
14212
17334
6410
17335
17336
14213
15777
11079
14214
6411
15778
6412
11080
9539
7965
6413
4882
14215
7966
14216
9540
11081
17337
17338
12649
4883
4884
14217
17339
7967
7968
17340
14218
14219
4885
4886
11082
12650
9541
7969
4887
12651
3313
14220
12652
6414
7970
7971
4888
15779
12653
15780
3314
3315
12654
14221
14222
9542
15781
17341
14223
12655
11083
17342
12656
7972
12657
4889
12658
4890
11084
7973
14224
17343
12659
4891
15782
7974
6415
14225
15783
9543
15784
15785
17344
14226
15786
15787
7975
14227
6416
6417
3316
6418
7976
15788
3317
11085
3318
14228
7977
12660
14229
4892
14230
3319
6419
17345
4893
3320
11086
7978
6420
17346
14231
4894
17347
11087
12661
12662
6421
14232
9544
4895
11088
15789
14233
6422
14234
15790
3321
11089
9545
7979
4896
15791
3322
12663
15792
14235
6423
6424
9546
11090
9547
4897
11091
17348
12664
14236
7980
14237
12665
15793
15794
12666
15795
15796
9548
15797
17349
11092
4898
3323
4899
11093
3324
9549
14238
6425
14239
17350
17351
79

In [292]:
#open with json - raw_truth_train

#raw_truth_train=[]
#with open('../clickbait_dataset/truth_train.jsonl','rb') as raw_truth_file: #opening file with binary(rb) mode.
#    for x in raw_truth_file:
#        raw_truth_train.append(json.loads(x))
#raw_truth_train[7]


In [293]:
#raw_instance_train_origin = [json.loads(line) for line in open("../clickbait_dataset/instances_train.jsonl")]
#raw_truth_train = [json.loads(line) for line in open("../clickbait_dataset/truth_train.jsonl")]

In [294]:
#print(len(raw_instance_train_origin))

In [295]:
raw_instance_train_origin = []
with open('../clickbait_dataset/instances_train.jsonl', 'r') as f:
    for line in f:
        dic = json.loads(line)
        raw_instance_train_origin.append(dic)
            
print(len(raw_instance_train_origin))

17581


In [296]:
#separate into two list (clickbait/ non_clickbait)
clickbait_train = []
non_clickbait_train = []
for x in range(0, len(raw_truth_train)):
    id_ = int(raw_truth_train[x]['id'])
    if raw_truth_train[x]['truthClass'] == 'clickbait':
        temp_dict = raw_instance_train_origin[id_].update(raw_truth_train[x])
        clickbait_train.append(raw_instance_train_origin[id_])
    else:
        temp_dict = raw_instance_train_origin[id_].update(raw_truth_train[x])
        non_clickbait_train.append(raw_instance_train_origin[id_])

In [297]:
clickbait_train[0]

{'id': '2',
 'postMedia': [],
 'postText': ['U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.'],
 'postTimestamp': 'Fri Jun 12 23:36:05 +0000 2015',
 'targetCaptions': ['US to vote for Ali in FIFA election and not Blatter',
  'US to vote for Ali in FIFA election and not Blatter',
  "FILE - This Oct. 10, 2014, file photo shows Sunil Gulati, president of the United States Soccer Federation, during a press conference in Bristol, Conn. The United States says it will vote for Jordan's Prince Ali bin Al-Hussein for FIFA president Friday, May 29, 2015 and not for incumbent Sepp Blatter. (AP Photo/Elise Amendola, File)"],
 'targetDescription': "A U.S. Senator's scathing letter questioned U.S. Soccer's inadequate handling of Solo's domestic violence charges. It's time for Sunil Gulati to respond.",
 'targetKeywords': '',
 'targetParagraphs': ["WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's concerns 

In [298]:
non_clickbait_train[0]

{'id': '0',
 'postMedia': [],
 'postText': ["Apple's iOS 9 'App thinning' feature will give your phone's storage a boost"],
 'postTimestamp': 'Tue Jun 09 16:31:10 +0000 2015',
 'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The Ap

In [299]:
#filter out empty postText dic
raw_instance_train = []
for x in range(0, len(raw_instance_train_origin)):
    if x == 2196:
        print(x, raw_instance_train_origin[x]['postText'][0], len(raw_instance_train_origin[x]['postText'][0]))
    if len(raw_instance_train_origin[x]['postText'][0]) > 0:       
        raw_instance_train.append(raw_instance_train_origin[x])
    else:
        print(x, raw_instance_train_origin[x]['id'],'postText',raw_instance_train_origin[x]['postText'])
print(len(raw_instance_train))

2196  0
2196 2196 postText ['']
2213 2213 postText ['']
2217 2217 postText ['']
2235 2235 postText ['']
2238 2238 postText ['']
2272 2272 postText ['']
3781 3781 postText ['']
3824 3824 postText ['']
3830 3830 postText ['']
3865 3865 postText ['']
5292 5292 postText ['']
5322 5322 postText ['']
5347 5347 postText ['']
5372 5372 postText ['']
5381 5381 postText ['']
6766 6766 postText ['']
6829 6829 postText ['']
6835 6835 postText ['']
6855 6855 postText ['']
6882 6882 postText ['']
6926 6926 postText ['']
8424 8424 postText ['']
8474 8474 postText ['']
8539 8539 postText ['']
8904 8904 postText ['']
9925 9925 postText ['']
9941 9941 postText ['']
9981 9981 postText ['']
11527 11527 postText ['']
11635 11635 postText ['']
13163 13163 postText ['']
13171 13171 postText ['']
13204 13204 postText ['']
13206 13206 postText ['']
13233 13233 postText ['']
14695 14695 postText ['']
14704 14704 postText ['']
14750 14750 postText ['']
14760 14760 postText ['']
14780 14780 postText ['']
16251 16

In [300]:
raw_instance_train[0]['truthClass']

'no-clickbait'

In [301]:
label = []
for x in range(0, len(raw_instance_train)):
    if raw_instance_train[x]['truthClass'] == 'clickbait':
        label.append(1)
    elif raw_instance_train[x]['truthClass'] == 'no-clickbait':
        label.append(0)
    else:
        print('what')
#print(label)

In [302]:
id_labels= [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]

In [258]:
if label == id_labels:
    print('true')
else:
    print("no")
print(len(label))
print(len(id_labels))

no
17535
17535


In [303]:
(raw_instance_train[1976])
for x in raw_truth_train:
    if x['id'] == '1976':
        print(x)

{'truthMedian': 0.0, 'truthMean': 0.13333333332, 'truthMode': 0.0, 'truthClass': 'no-clickbait', 'truthJudgments': [0.3333333333, 0.0, 0.3333333333, 0.0, 0.0], 'id': '1976'}


In [304]:
for x in range(0, len(label)):
    if label[x] != id_labels[x]:
        print('x:', x, raw_instance_train[x]['id'])

x: 1976 1976
x: 1978 1978
x: 1982 1982
x: 1984 1984
x: 1986 1986
x: 1991 1991
x: 1993 1993
x: 1994 1994
x: 1996 1996
x: 2005 2005
x: 2007 2007
x: 2009 2009
x: 2014 2014
x: 2022 2022
x: 2023 2023
x: 2027 2027
x: 2029 2029
x: 2032 2032
x: 2035 2035
x: 2036 2036
x: 2038 2038
x: 2040 2040
x: 2041 2041
x: 2047 2047
x: 2050 2050
x: 2057 2057
x: 2059 2059
x: 2060 2060
x: 2062 2062
x: 2071 2071
x: 2072 2072
x: 2073 2073
x: 2075 2075
x: 2078 2078
x: 2083 2083
x: 2084 2084
x: 2090 2090
x: 2093 2093
x: 2095 2095
x: 2096 2096
x: 2099 2099
x: 2104 2104
x: 2107 2107
x: 2115 2115
x: 2116 2116
x: 2122 2122
x: 2126 2126
x: 2127 2127
x: 2129 2129
x: 2130 2130
x: 2132 2132
x: 2133 2133
x: 2136 2136
x: 2149 2149
x: 2161 2161
x: 2162 2162
x: 2163 2163
x: 2166 2166
x: 2171 2171
x: 2173 2173
x: 2176 2176
x: 2179 2179
x: 2180 2180
x: 2181 2181
x: 2185 2185
x: 2187 2187
x: 2188 2188
x: 2190 2190
x: 2192 2192
x: 2196 2197
x: 2206 2207
x: 2207 2208
x: 2209 2210
x: 2210 2211
x: 2212 2214
x: 2216 2219
x: 2217 2220

x: 4911 4921
x: 4913 4923
x: 4915 4925
x: 4918 4928
x: 4920 4930
x: 4927 4937
x: 4932 4942
x: 4934 4944
x: 4935 4945
x: 4937 4947
x: 4940 4950
x: 4942 4952
x: 4944 4954
x: 4945 4955
x: 4947 4957
x: 4950 4960
x: 4953 4963
x: 4955 4965
x: 4958 4968
x: 4961 4971
x: 4963 4973
x: 4968 4978
x: 4969 4979
x: 4971 4981
x: 4972 4982
x: 4977 4987
x: 4979 4989
x: 4985 4995
x: 4987 4997
x: 4989 4999
x: 4991 5001
x: 4992 5002
x: 4995 5005
x: 4996 5006
x: 5000 5010
x: 5003 5013
x: 5005 5015
x: 5009 5019
x: 5011 5021
x: 5012 5022
x: 5015 5025
x: 5016 5026
x: 5019 5029
x: 5020 5030
x: 5021 5031
x: 5022 5032
x: 5025 5035
x: 5027 5037
x: 5029 5039
x: 5030 5040
x: 5033 5043
x: 5035 5045
x: 5036 5046
x: 5048 5058
x: 5050 5060
x: 5054 5064
x: 5055 5065
x: 5058 5068
x: 5060 5070
x: 5061 5071
x: 5062 5072
x: 5066 5076
x: 5069 5079
x: 5070 5080
x: 5075 5085
x: 5077 5087
x: 5080 5090
x: 5081 5091
x: 5082 5092
x: 5083 5093
x: 5084 5094
x: 5086 5096
x: 5102 5112
x: 5104 5114
x: 5105 5115
x: 5108 5118
x: 5112 5122

x: 7344 7365
x: 7345 7366
x: 7347 7368
x: 7348 7369
x: 7354 7375
x: 7355 7376
x: 7356 7377
x: 7357 7378
x: 7359 7380
x: 7360 7381
x: 7362 7383
x: 7364 7385
x: 7365 7386
x: 7366 7387
x: 7367 7388
x: 7370 7391
x: 7373 7394
x: 7374 7395
x: 7379 7400
x: 7384 7405
x: 7390 7411
x: 7392 7413
x: 7393 7414
x: 7394 7415
x: 7396 7417
x: 7401 7422
x: 7403 7424
x: 7406 7427
x: 7409 7430
x: 7413 7434
x: 7415 7436
x: 7417 7438
x: 7418 7439
x: 7420 7441
x: 7421 7442
x: 7422 7443
x: 7425 7446
x: 7427 7448
x: 7430 7451
x: 7435 7456
x: 7437 7458
x: 7439 7460
x: 7440 7461
x: 7443 7464
x: 7444 7465
x: 7448 7469
x: 7449 7470
x: 7451 7472
x: 7452 7473
x: 7453 7474
x: 7454 7475
x: 7456 7477
x: 7460 7481
x: 7461 7482
x: 7462 7483
x: 7465 7486
x: 7470 7491
x: 7474 7495
x: 7480 7501
x: 7482 7503
x: 7483 7504
x: 7484 7505
x: 7492 7513
x: 7495 7516
x: 7497 7518
x: 7498 7519
x: 7499 7520
x: 7501 7522
x: 7515 7536
x: 7519 7540
x: 7520 7541
x: 7532 7553
x: 7538 7559
x: 7543 7564
x: 7548 7569
x: 7551 7572
x: 7552 7573

x: 10187 10215
x: 10188 10216
x: 10190 10218
x: 10191 10219
x: 10195 10223
x: 10196 10224
x: 10197 10225
x: 10206 10234
x: 10215 10243
x: 10218 10246
x: 10221 10249
x: 10222 10250
x: 10225 10253
x: 10226 10254
x: 10229 10257
x: 10231 10259
x: 10232 10260
x: 10233 10261
x: 10239 10267
x: 10241 10269
x: 10243 10271
x: 10246 10274
x: 10248 10276
x: 10249 10277
x: 10251 10279
x: 10252 10280
x: 10254 10282
x: 10258 10286
x: 10259 10287
x: 10260 10288
x: 10261 10289
x: 10263 10291
x: 10264 10292
x: 10268 10296
x: 10275 10303
x: 10277 10305
x: 10279 10307
x: 10280 10308
x: 10282 10310
x: 10283 10311
x: 10284 10312
x: 10285 10313
x: 10289 10317
x: 10290 10318
x: 10291 10319
x: 10292 10320
x: 10293 10321
x: 10298 10326
x: 10303 10331
x: 10305 10333
x: 10307 10335
x: 10308 10336
x: 10309 10337
x: 10314 10342
x: 10315 10343
x: 10316 10344
x: 10320 10348
x: 10322 10350
x: 10325 10353
x: 10326 10354
x: 10328 10356
x: 10330 10358
x: 10334 10362
x: 10335 10363
x: 10338 10366
x: 10339 10367
x: 10341 1

x: 12970 13000
x: 12972 13002
x: 12974 13004
x: 12976 13006
x: 12981 13011
x: 12982 13012
x: 12984 13014
x: 12991 13021
x: 12993 13023
x: 12996 13026
x: 12999 13029
x: 13003 13033
x: 13013 13043
x: 13018 13048
x: 13021 13051
x: 13025 13055
x: 13031 13061
x: 13035 13065
x: 13039 13069
x: 13044 13074
x: 13045 13075
x: 13047 13077
x: 13048 13078
x: 13053 13083
x: 13054 13084
x: 13055 13085
x: 13056 13086
x: 13058 13088
x: 13060 13090
x: 13064 13094
x: 13069 13099
x: 13072 13102
x: 13077 13107
x: 13078 13108
x: 13079 13109
x: 13081 13111
x: 13083 13113
x: 13085 13115
x: 13087 13117
x: 13095 13125
x: 13098 13128
x: 13104 13134
x: 13111 13141
x: 13114 13144
x: 13117 13147
x: 13119 13149
x: 13120 13150
x: 13121 13151
x: 13123 13153
x: 13124 13154
x: 13125 13155
x: 13126 13156
x: 13127 13157
x: 13132 13162
x: 13133 13164
x: 13136 13167
x: 13137 13168
x: 13140 13172
x: 13149 13181
x: 13153 13185
x: 13154 13186
x: 13157 13189
x: 13158 13190
x: 13164 13196
x: 13167 13199
x: 13173 13207
x: 13180 1

x: 15626 15666
x: 15627 15667
x: 15629 15669
x: 15631 15671
x: 15635 15675
x: 15636 15676
x: 15638 15678
x: 15640 15680
x: 15642 15682
x: 15643 15683
x: 15646 15686
x: 15648 15688
x: 15649 15689
x: 15650 15690
x: 15653 15693
x: 15654 15694
x: 15655 15695
x: 15658 15698
x: 15659 15699
x: 15660 15700
x: 15661 15701
x: 15663 15703
x: 15666 15706
x: 15669 15709
x: 15673 15713
x: 15677 15717
x: 15681 15721
x: 15684 15724
x: 15685 15725
x: 15686 15726
x: 15691 15731
x: 15692 15732
x: 15693 15733
x: 15699 15739
x: 15701 15741
x: 15702 15742
x: 15705 15745
x: 15710 15750
x: 15711 15751
x: 15716 15756
x: 15719 15759
x: 15721 15761
x: 15723 15763
x: 15724 15764
x: 15725 15765
x: 15731 15771
x: 15734 15774
x: 15738 15778
x: 15739 15779
x: 15740 15780
x: 15741 15781
x: 15745 15785
x: 15748 15788
x: 15751 15791
x: 15753 15793
x: 15760 15800
x: 15764 15804
x: 15767 15807
x: 15769 15809
x: 15772 15812
x: 15775 15815
x: 15779 15819
x: 15781 15821
x: 15782 15822
x: 15783 15823
x: 15786 15826
x: 15787 1

In [305]:
#Lexicon (Word) Count
targetTitleCountWords = []
for x in range(0, len(raw_instance_train)):
    targetTitleCountWords.append(textstat.lexicon_count(raw_instance_train[x]['targetTitle']))
targetTitleCountWords

[17,
 8,
 10,
 11,
 6,
 11,
 10,
 11,
 12,
 14,
 10,
 12,
 12,
 23,
 10,
 14,
 9,
 24,
 7,
 10,
 12,
 10,
 7,
 5,
 11,
 14,
 8,
 11,
 10,
 7,
 14,
 8,
 11,
 10,
 9,
 11,
 7,
 19,
 11,
 1,
 9,
 13,
 12,
 3,
 10,
 299,
 15,
 9,
 10,
 295,
 11,
 21,
 7,
 7,
 10,
 11,
 10,
 9,
 10,
 10,
 12,
 8,
 19,
 10,
 307,
 295,
 8,
 4,
 5,
 9,
 9,
 4,
 11,
 10,
 276,
 9,
 21,
 9,
 8,
 8,
 11,
 8,
 18,
 6,
 20,
 8,
 12,
 15,
 10,
 1,
 12,
 9,
 20,
 10,
 8,
 13,
 13,
 11,
 13,
 6,
 12,
 12,
 10,
 11,
 12,
 10,
 8,
 11,
 12,
 10,
 10,
 11,
 11,
 9,
 8,
 10,
 14,
 7,
 8,
 14,
 10,
 8,
 20,
 15,
 11,
 10,
 6,
 14,
 9,
 12,
 10,
 9,
 13,
 17,
 7,
 10,
 12,
 6,
 18,
 17,
 8,
 7,
 10,
 13,
 16,
 17,
 12,
 13,
 301,
 7,
 4,
 11,
 12,
 10,
 12,
 18,
 9,
 12,
 7,
 5,
 12,
 4,
 8,
 11,
 9,
 9,
 9,
 13,
 13,
 2,
 10,
 8,
 7,
 11,
 10,
 10,
 20,
 10,
 10,
 9,
 7,
 10,
 11,
 25,
 11,
 36,
 11,
 12,
 3,
 8,
 7,
 9,
 9,
 18,
 303,
 11,
 16,
 12,
 11,
 9,
 7,
 10,
 13,
 5,
 10,
 10,
 7,
 20,
 11,
 12,
 6,
 9,
 7,
 8,


In [306]:
#Grade Level reading difficulty
#Coleman-Liau index, Gunning FOG Formula, Linsear Write Formula
targetTitle_coleman_liau_index = []
targetTitle_gunning_fog = []
targetTitle_linsear_write_formula = []
for x in range(0, len(raw_instance_train)):
    targetTitle_coleman_liau_index.append(textstat.coleman_liau_index(raw_instance_train[x]['targetTitle']))
    targetTitle_gunning_fog.append(textstat.gunning_fog(raw_instance_train[x]['targetTitle']))
    targetTitle_linsear_write_formula.append(textstat.linsear_write_formula(raw_instance_train[x]['targetTitle']))


In [307]:
print(raw_instance_train[0]['targetTitle'])
textstat.coleman_liau_index(raw_instance_train[0]['targetTitle'])


Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


12.12

In [308]:
print(raw_instance_train[0]['targetTitle'])
textstat.gunning_fog(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


20.564705882352943

In [309]:
print(raw_instance_train[0]['targetTitle'])
textstat.linsear_write_formula(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


6.5

In [310]:
#clickbait_train, emoji
def pos_1gram(data, title):
    postag_words = word_tokenize(data.get(title)[0])
    return nltk.pos_tag(postag_words)

pos_tag_list = []
#for single_data in clickbait_train:
    #print(pos_2gram(single_data, "postText"))

In [311]:
pos_1gram(clickbait_train[0], "postText")

[('U.S.', 'NNP'),
 ('Soccer', 'NNP'),
 ('should', 'MD'),
 ('start', 'VB'),
 ('answering', 'VBG'),
 ('tough', 'JJ'),
 ('questions', 'NNS'),
 ('about', 'IN'),
 ('Hope', 'NNP'),
 ('Solo', 'NNP'),
 (',', ','),
 ('@', 'NNP'),
 ('eric_adelson', 'NN'),
 ('writes', 'NNS'),
 ('.', '.')]

In [312]:
#calculate tags in a data
def cal_pos_1grams_tag(data, title, tag):
    i = 0
    pos_list = []
    postag_words = word_tokenize(data.get(title)[0])
    pos_list = nltk.pos_tag(postag_words)
    for tags in pos_list:
        #print("tags: ", tags)
        if tags[1] == tag:
            i+=1
    return i   

In [313]:
cal_pos_1grams_tag(clickbait_train[0], "postText", "NNP")
cal_pos_1grams_tag_list = []
for x in range(0, len(raw_instance_train)):
    cal_pos_1grams_tag_list.append(cal_pos_1grams_tag(raw_instance_train[x], "postText", "NNP"))
cal_pos_1grams_tag_list

[1,
 3,
 5,
 2,
 0,
 0,
 2,
 0,
 4,
 4,
 1,
 1,
 2,
 4,
 1,
 9,
 4,
 0,
 0,
 3,
 4,
 0,
 3,
 4,
 4,
 0,
 5,
 1,
 1,
 1,
 1,
 0,
 0,
 5,
 1,
 2,
 0,
 10,
 3,
 2,
 1,
 7,
 2,
 2,
 3,
 1,
 2,
 2,
 4,
 1,
 1,
 2,
 3,
 2,
 2,
 2,
 0,
 3,
 2,
 4,
 7,
 0,
 0,
 1,
 8,
 2,
 10,
 0,
 4,
 4,
 0,
 3,
 1,
 1,
 1,
 5,
 6,
 1,
 1,
 5,
 0,
 3,
 1,
 1,
 4,
 5,
 3,
 3,
 2,
 4,
 1,
 3,
 4,
 3,
 6,
 3,
 2,
 4,
 1,
 2,
 1,
 3,
 6,
 4,
 2,
 2,
 3,
 8,
 3,
 11,
 4,
 2,
 5,
 1,
 1,
 7,
 4,
 4,
 1,
 1,
 3,
 0,
 1,
 8,
 3,
 0,
 2,
 2,
 3,
 2,
 6,
 3,
 1,
 3,
 2,
 2,
 5,
 8,
 5,
 1,
 4,
 2,
 6,
 0,
 1,
 2,
 1,
 6,
 3,
 1,
 3,
 4,
 2,
 1,
 3,
 2,
 3,
 2,
 5,
 8,
 1,
 0,
 7,
 2,
 4,
 3,
 4,
 3,
 7,
 13,
 5,
 0,
 3,
 6,
 1,
 2,
 4,
 2,
 2,
 5,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 8,
 2,
 3,
 2,
 7,
 2,
 2,
 4,
 2,
 1,
 9,
 2,
 1,
 3,
 2,
 3,
 4,
 0,
 3,
 1,
 0,
 2,
 0,
 5,
 7,
 4,
 2,
 0,
 0,
 4,
 0,
 5,
 2,
 3,
 3,
 2,
 3,
 10,
 2,
 7,
 5,
 3,
 3,
 8,
 3,
 7,
 3,
 8,
 3,
 2,
 1,
 2,
 0,
 2,
 5,
 1,
 0,
 0,
 4,
 0,
 0,

In [314]:
#yutarochan's code
def pos_2gram(title, p1, p2):
    pos_list = pos_tag(word_tokenize(title))
    return sum(map(lambda x: x[0][1] == p1 and x[1][1] == p2, zip(pos_list[:-1], pos_list[1:])))

In [315]:
clickbait_train[0]['targetTitle']

'U.S. Soccer should start answering tough questions about Hope Solo'

In [316]:
pos_2gram(clickbait_train[0]['targetTitle'], 'NNP', 'NNP')

2

In [317]:
#revised pos_2grams
def pos_2gram_sum(title, p1, p2):
    pos_2g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2g_num += 1
    return pos_2g_num

In [318]:
pos_2gram_NNP_NNP = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_NNP.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNP'))
pos_2gram_NNP_NNP

[0,
 2,
 2,
 1,
 0,
 2,
 5,
 0,
 3,
 1,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 1,
 4,
 4,
 3,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 8,
 3,
 0,
 0,
 32,
 0,
 3,
 3,
 30,
 0,
 0,
 0,
 3,
 0,
 4,
 6,
 0,
 4,
 0,
 6,
 3,
 0,
 5,
 32,
 34,
 5,
 0,
 2,
 0,
 0,
 0,
 0,
 4,
 77,
 2,
 2,
 5,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 5,
 2,
 0,
 7,
 3,
 1,
 1,
 1,
 0,
 4,
 2,
 4,
 0,
 0,
 4,
 7,
 0,
 1,
 1,
 4,
 3,
 6,
 6,
 1,
 0,
 2,
 4,
 0,
 1,
 9,
 1,
 0,
 3,
 7,
 0,
 0,
 3,
 1,
 5,
 0,
 5,
 6,
 4,
 4,
 1,
 2,
 0,
 0,
 5,
 5,
 1,
 3,
 0,
 1,
 2,
 7,
 7,
 7,
 2,
 0,
 3,
 28,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 1,
 4,
 1,
 0,
 3,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 4,
 3,
 1,
 0,
 3,
 3,
 4,
 2,
 1,
 9,
 1,
 0,
 0,
 4,
 2,
 0,
 5,
 2,
 0,
 3,
 6,
 34,
 1,
 0,
 0,
 4,
 5,
 4,
 3,
 3,
 1,
 4,
 0,
 4,
 1,
 0,
 2,
 0,
 2,
 1,
 0,
 5,
 0,
 0,
 5,
 1,
 44,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 4,
 1,
 0,
 0,
 4,
 0,
 2,
 3,
 6,
 0,
 3,
 31,
 3,
 0,
 0,
 3,
 0,
 0,
 0,
 1,
 1,

In [319]:
#check pos_2grams
def pos_2gram_check(title, p1, p2):
    pos_2gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2gram_list.append([pos_z[0], pos_z[1]])
    return pos_2gram_list

In [320]:
print(pos_2gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP'))

[[('U.S.', 'NNP'), ('Soccer', 'NNP')], [('Hope', 'NNP'), ('Solo', 'NNP')]]


In [321]:
#revised pos_3grams
def pos_3gram_sum(title, p1, p2, p3):
    pos_3g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3g_num += 1
    return pos_3g_num

In [322]:
pos_3gram_NNP_NNP_NNP = []
for x in range(0, len(raw_instance_train)):
    pos_3gram_NNP_NNP_NNP.append(pos_3gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNP', 'NNP'))
len(pos_3gram_NNP_NNP_NNP)
print(pos_3gram_NNP_NNP_NNP)

[0, 1, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 12, 0, 1, 2, 9, 0, 0, 0, 1, 0, 2, 4, 0, 3, 0, 2, 2, 0, 3, 9, 14, 4, 0, 1, 0, 0, 0, 0, 1, 49, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 5, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 3, 4, 0, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 0, 0, 5, 0, 0, 1, 5, 0, 0, 1, 0, 4, 0, 2, 4, 2, 0, 0, 1, 0, 0, 3, 1, 0, 1, 0, 0, 0, 4, 4, 3, 1, 0, 1, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 2, 1, 1, 0, 6, 0, 0, 0, 3, 1, 0, 3, 1, 0, 2, 3, 12, 0, 0, 0, 1, 3, 3, 2, 2, 0, 3, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 26, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 2, 3, 0, 1, 12, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 7, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 3, 1, 1, 0, 4, 6, 2, 0, 0, 0, 0, 3, 0, 0, 16, 2, 1, 0, 0, 6, 0, 0, 0, 0, 1, 0, 0, 6, 2, 27, 0, 2, 0, 3, 0, 0, 1, 0, 0, 0, 0

In [323]:
#check pos_3grams
def pos_3gram_check(title, p1, p2, p3):
    pos_3gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3gram_list.append([pos_z[0], pos_z[1],pos_z[2]])
    return pos_3gram_list

In [324]:
print(pos_3gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP', 'NNP'))

[]


In [325]:
###Oct 6, Ethan's new features
#Some methods I need
#assorted cleaning
stopWords = stopwords.words('english')

def stem_tokens(tokens, ps):
    stemmed = []
    for item in tokens:
        stemmed.append(ps.stem(item))
    return stemmed
ps = PorterStemmer()

def lemmatize_tokens(tokens, wnl):
    lemmatized = []
    for item in tokens:
        lemmatized.append(wnl.lemmatize(item))
    return lemmatized
wnl = WordNetLemmatizer()

def tokensClean(text):
    tokens = word_tokenize(text)
    tokens = stem_tokens(tokens, ps)
    tokens = lemmatize_tokens(tokens, wnl)
    tokens = [w for w in tokens if not w in stopWords]
    tokens = [c for c in tokens if c not in punctuation]
    return (tokens)

def doc_punc_parser(document):
    #print(document)
    #print(porter_stemmer.stem(document))
    #print(stem_doc)
    tokens = word_tokenize(document)
    non_empty_tokens = [x for x in tokens if x != []]
    #clean_tokens = [c for c in non_empty_tokens if c not in punctuation and stop_words]
    #non_apos_tokens = [c.replace("'", "") for c in non_empty_tokens]
    return non_empty_tokens

def tweet_Tokenizer(document):
    tknzr = TweetTokenizer()
    #tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
    
    return [c for c in tknzr.tokenize(document) if c not in punctuation]

In [326]:
target_para_list = []
for n in clickbait_train[0]['targetParagraphs']:
    target_para_ = doc_punc_parser(n)
    #print(type(target_para_list))
    #print(target_para_)
    target_para_list = target_para_list + target_para_
    #print(n)
    #print(nltk.pos_tag(n))

target_para_tags = nltk.pos_tag(target_para_list)

In [327]:
def count_tags (tags_list):
    tag_fd = nltk.FreqDist(tag for (word, tag) in tags_list)
    return (tag_fd.most_common())

In [328]:
print(count_tags(target_para_tags))

#for x in range(0, len(raw_instance_train)):


[('NN', 115), ('NNP', 90), ('DT', 88), ('IN', 87), ('JJ', 44), ('VBZ', 41), ('RB', 40), ('.', 31), ('VB', 30), (',', 27), ('NNS', 24), ('TO', 23), ('CC', 20), ('POS', 16), ('VBD', 15), ('VBG', 14), ('PRP', 13), ('VBN', 12), ('PRP$', 10), ("''", 10), ('``', 9), ('VBP', 6), ('MD', 6), ('JJR', 5), ('WP', 4), ('WRB', 3), (':', 3), ('CD', 3), ('EX', 3), ('JJS', 2), ('(', 2), ('WDT', 2), (')', 2), ('RBR', 2), ('RBS', 1), ('RP', 1), ('NNPS', 1)]


In [329]:
def findtags(tag_prefix, tagged_text):
    cfd = nltk.ConditionalFreqDist((tag, word) for (word, tag) in tagged_text
                                  if tag.startswith(tag_prefix))
    return dict((tag, cfd[tag].most_common(5)) for tag in cfd.conditions())

In [330]:
findtags("NN",target_para_tags)

{'NN': [('violence', 6),
  ('lot', 4),
  ('incident', 3),
  ('explanation', 3),
  ('game', 3)],
 'NNP': [('Solo', 15),
  ('U.S.', 11),
  ('Soccer', 10),
  ('Gulati', 5),
  ('Hope', 4)],
 'NNPS': [('Americans', 1)],
 'NNS': [('women', 2),
  ('questions', 2),
  ('months', 2),
  ('concerns', 1),
  ('values', 1)]}

In [331]:
##1. Number of Characters. 
def count_array_char(text):
    if text != []:
        tokens = [doc_punc_parser(t) for t in text]
        return sum([sum([len(t) for t in token])for token in tokens])
    else:
        return -1
    
def count_string_char(text):
    if text != "":
        tokens = doc_punc_parser(text)
        return sum([len(t) for t in tokens])
    else:
        return -1

def count_tweet_array_char(text):
    if text != []:
        tokens = [tweet_Tokenizer(t) for t in text]
        return sum([sum([len(t) for t in token])for token in tokens])
    else:
        return -1
    
def count_tweet_string_char(text):
    if text != "":
        tokens = tweet_Tokenizer(text)
        return sum([len(t) for t in tokens])
    else:
        return -1

In [332]:
##2. Number of Words. (include punctuation)
def count_array_word(text):
    tokens = [doc_punc_parser(t) for t in text]
    return sum([len(token) for token in tokens])
def count_string_word(text):
    token = doc_punc_parser(text)
    return (len(token))

#3. Numbers of words(exclude punctuation, and '@'/'#')
def count_tweet_array_word(text):
    if text != []:
        tokens = [tweet_Tokenizer(t) for t in text]
        return sum([len(token) for token in tokens])
    else:
        return -1
def count_tweet_string_word(text):
    if text != "":
        token = tweet_Tokenizer(text)
        return len(token)
    else:
        return -1

In [333]:
##3. Common Words Between Article Keywords and Others.
#parsing the targetKeywords.
#Made a new function for parsing since targetKeywords are separated by "," 
def keyWord_parse_(keywords):
    keyList = keywords.split(",")
    #keyList = stem_tokens(keyList, ps)
    #keyList = lemmatize_tokens(keyList, wnl)  
    if (keyList) != ['']:
        return keyList
    else:
        #print("keyList",keyList)
        return -1
    
import re
def keyWordFrequency_string_(keyList, strings):
    counter = 0
    strings = strings.lower()
    if keyList != -1 and not (strings.isspace()) and strings != '':
        for word in keyList:
            pattern = re.compile(r'\b{}\b'.format(word), re.I) # case insensitive
            num = len(pattern.findall(strings))
            counter += num
        if count_string_word(strings) == 0:
            print("11:",strings,",",len(strings))
        return counter/count_string_word(strings)
    else:
        return -1

def keyWordFrequency_array_(keyList, array):
    counter = 0
    if keyList != -1 and not (all('' == s or s.isspace() for s in array)) and array != []:
        for strings in array:
            for word in keyList:
                pattern = re.compile(r'\b{}\b'.format(word), re.I) # case insensitive
                if pattern.findall(strings):
                    num = len(pattern.findall(strings))
                    counter += num
        if count_array_word(array) == 0:
            print('array:',array)
        return counter/count_array_word(array)
    else:
        return -1

In [334]:
##4.Formal, and Informal English Word Ratio in postText
def formal_word_preprocess(text):
    if text != [] and not (all('' == s or s.isspace() for s in text)):
        stoplist = list('for a of the and to in'.split())
        formal_list = lemmatize_tokens(tweet_Tokenizer(text), wnl)
        formal_list = [f.replace("'s", "") for f in formal_list]
        formal_list = list(filter((lambda x: x not in punctuation and x not in stoplist), formal_list))
        return formal_list
    else:
        return -1

def count_formal_ratio(text):
    text = text[0]
    counter = 0
    formal_list = formal_word_preprocess(text)
    if formal_list != -1:
        for f in formal_list:
            definitions = wn.synsets(f)
            if len(definitions) > 0:
                counter += 1
        formal_ratio = counter / len(formal_word_preprocess(text))
        #print(formal_word_preprocess(text))
        return formal_ratio
    else:
        return -1

In [335]:
#5. clickbait has more '#' and '@'
def paragraphs_tags_parse(paragraphs):
    if len(paragraphs) != 0:
        return [doc_punc_parser(para) for para in paragraphs]
    else:
        return -1
def tags_parse(tagwords):
    tagList = tagwords.split(",")
    if tagList != ['']:
        return tagList
    else:
        print("keyList",tagList)
        return -1
def tagFrequency_array(keyList, array):  
    if keyList != -1 and array != -1:
        counter = 0
        for key in keyList:
            for strings in array:
                for s in strings:
                    if s == key:
                        #print(strings, key)
                        counter += 1
        return counter
    else:
        return -1

In [336]:
#1a. the number of characters in post’s title.
postTitle_char_count = []
for x in range(0, len(raw_instance_train)):
    postTitle_char_count.append(count_array_char(raw_instance_train[x]['postText']))
print(postTitle_char_count)

[63, 87, 78, 52, 76, 54, 77, 57, 70, 67, 62, 55, 68, 73, 58, 121, 59, 58, 42, 83, 74, 38, 70, 48, 54, 54, 57, 43, 57, 52, 76, 21, 95, 99, 44, 53, 45, 98, 93, 96, 41, 80, 54, 79, 62, 77, 67, 44, 94, 73, 52, 76, 66, 49, 44, 58, 48, 67, 62, 100, 70, 37, 61, 64, 76, 80, 65, 75, 89, 84, 70, 29, 72, 67, 97, 43, 43, 55, 33, 60, 42, 52, 42, 95, 68, 80, 60, 88, 47, 41, 72, 40, 78, 50, 53, 59, 54, 53, 77, 71, 54, 78, 85, 69, 57, 54, 91, 76, 51, 68, 119, 55, 58, 47, 52, 87, 72, 80, 47, 70, 69, 48, 74, 56, 72, 45, 42, 59, 68, 67, 125, 71, 81, 71, 44, 79, 65, 120, 96, 47, 81, 63, 82, 62, 73, 78, 80, 72, 79, 82, 64, 69, 61, 74, 78, 75, 60, 68, 68, 76, 70, 56, 86, 51, 97, 99, 87, 65, 81, 98, 61, 60, 95, 72, 71, 64, 71, 41, 53, 66, 39, 107, 73, 88, 56, 89, 78, 60, 71, 53, 24, 51, 65, 74, 73, 77, 99, 51, 85, 46, 35, 71, 71, 60, 77, 50, 83, 73, 52, 61, 26, 99, 97, 57, 69, 74, 65, 55, 35, 72, 70, 63, 118, 46, 61, 101, 59, 61, 75, 74, 85, 60, 56, 56, 96, 75, 66, 66, 69, 67, 64, 100, 54, 61, 35, 64, 100, 6

In [337]:
#1b. the number of characters in article’s title.
targetTitle_char_count = []
for x in range(0, len(raw_instance_train)):
    targetTitle_char_count.append(count_string_char(raw_instance_train[x]['targetTitle']))
print(targetTitle_char_count)

[87, 47, 57, 52, 29, 54, 67, 56, 52, 65, 62, 62, 66, 131, 57, 57, 59, 118, 42, 47, 64, 48, 47, 29, 58, 51, 55, 44, 38, 34, 64, 34, 67, 43, 53, 53, 43, 94, 53, 4, 41, 66, 54, 25, 62, 1631, 83, 44, 51, 1592, 52, 136, 48, 49, 44, 58, 46, 45, 62, 55, 62, 37, 98, 40, 1655, 1600, 53, 12, 22, 41, 51, 21, 73, 48, 1490, 43, 119, 55, 33, 32, 73, 39, 90, 37, 119, 37, 60, 85, 40, 6, 72, 40, 103, 50, 43, 63, 67, 52, 61, 34, 62, 50, 59, 59, 57, 52, 38, 47, 50, 55, 67, 55, 57, 48, 52, 62, 87, 29, 57, 60, 55, 50, 89, 79, 54, 45, 41, 66, 68, 50, 52, 57, 72, 96, 44, 58, 62, 51, 96, 91, 47, 39, 66, 77, 71, 73, 47, 72, 1620, 33, 22, 58, 61, 61, 68, 89, 43, 66, 36, 35, 69, 29, 44, 51, 56, 48, 41, 65, 65, 12, 61, 57, 30, 58, 36, 64, 82, 40, 53, 41, 39, 62, 66, 127, 56, 165, 64, 60, 20, 53, 24, 51, 43, 85, 1634, 58, 75, 51, 55, 61, 35, 50, 65, 38, 63, 49, 56, 96, 44, 60, 26, 56, 38, 34, 85, 73, 54, 53, 33, 1645, 54, 38, 90, 36, 40, 64, 99, 57, 49, 17, 37, 37, 43, 118, 44, 64, 56, 64, 1633, 76, 61, 100, 54, 6

In [338]:
#1c. the number of characters in article’s description.
targetDescription_char_count = []
for x in range(0, len(raw_instance_train)):
    targetDescription_char_count.append(count_string_char(raw_instance_train[x]['targetDescription']))
print(targetDescription_char_count)

[154, 140, 132, 100, 103, -1, 123, 143, 115, 135, 113, 134, 31, 159, 61, 83, 110, 162, 111, 109, 68, 106, 38, 74, 123, -1, 148, -1, -1, 92, 124, 90, 119, 136, -1, 103, 127, 145, 248, 133, 103, 145, 119, -1, 218, 86, 104, 147, 118, 30, 166, 153, 100, 158, 141, 113, 108, 73, 92, 195, 118, 28, 125, 257, 124, 69, 109, 76, 92, 51, 211, 114, 126, 152, 70, 85, 146, 134, 110, 126, -1, 173, 156, 125, 136, 105, 93, 139, 143, -1, 100, 87, 146, 114, 50, 397, 113, 256, 176, 131, 189, 240, -1, 175, 78, 81, 214, 93, -1, 256, 129, 118, 131, 299, 127, 121, -1, 25, 189, -1, 110, 30, 112, 131, 66, -1, 44, -1, 187, -1, 160, 135, 162, 96, 103, 102, 153, 114, 121, 190, 131, 126, 134, 367, 134, 95, 73, 138, 65, 111, 70, 129, -1, 117, 129, 182, 209, 31, 171, -1, 227, 131, 118, 219, 122, 104, 142, 122, -1, -1, 107, 99, 150, 181, 47, 109, 145, 124, 116, 113, -1, 46, 71, 149, 142, 160, 160, 123, 75, 130, 133, 190, -1, 74, 84, 74, 185, 83, 105, 125, 47, 131, 157, 133, 160, 115, 105, 150, 121, 133, 64, 128, 153, 1

In [339]:
#1d. the number of characters in article’s keywords.
targetKeywords_char_count = []
for x in range(0, len(raw_instance_train)):
    targetKeywords_char_count.append(count_tweet_array_char(raw_instance_train[x]['targetKeywords']))
print(targetKeywords_char_count)

[66, 111, 0, 106, 54, 0, 52, 0, 142, 0, 41, 33, 94, 14, 72, 15, 47, 80, 47, 0, 37, 0, 0, 56, 210, 0, 165, 0, 0, 0, 0, 0, 99, 45, 0, 53, 168, 26, 0, 0, 59, 25, 0, 0, 0, 89, 0, 134, 77, 87, 0, 116, 0, 290, 12, 0, 0, 63, 0, 72, 50, 65, 55, 0, 199, 95, 94, 38, 25, 84, 0, 133, 68, 0, 71, 84, 98, 133, 72, 45, 94, 0, 76, 75, 45, 0, 123, 107, 4, 0, 0, 0, 71, 85, 0, 19, 0, 0, 314, 72, 16, 0, 0, 0, 34, 0, 137, 25, 0, 0, 56, 26, 58, 0, 61, 46, 0, 0, 37, 0, 0, 28, 0, 0, 80, 0, 52, 0, 15, 0, 135, 94, 70, 14, 99, 35, 26, 0, 18, 14, 61, 116, 88, 0, 99, 42, 83, 0, 83, 46, 22, 83, 0, 63, 191, 66, 0, 94, 0, 0, 0, 190, 0, 0, 0, 0, 115, 51, 70, 0, 70, 89, 0, 0, 0, 109, 44, 0, 0, 101, 0, 37, 110, 109, 0, 27, 56, 98, 23, 61, 96, 0, 0, 13, 69, 38, 0, 74, 0, 0, 135, 305, 63, 4, 83, 33, 0, 56, 140, 79, 41, 0, 104, 0, 107, 0, 0, 119, 150, 79, 78, 0, 41, 120, 40, 0, 86, 89, 49, 0, 39, 0, 60, 96, 134, 15, 0, 124, 95, 30, 55, 0, 0, 112, 42, 103, 0, 141, 20, 48, 0, 67, 0, 17, 0, 53, 46, 49, 30, 0, 0, 57, 82, 120, 2

In [340]:
#1e. the number of characters in article’s captions.
targetCaptions_char_count = []
for x in range(0, len(raw_instance_train)):
    targetCaptions_char_count.append(count_array_char(raw_instance_train[x]['targetCaptions']))
print(targetCaptions_char_count)

[2330, 1879, 362, 875, 177, -1, -1, -1, 882, 145, 13, 7, -1, 87, 129, 141, 14, 1803, 211, 37, -1, -1, -1, 101345, 198, -1, 254, 289, 304, -1, 96, 14, 2385, 33405, 239, 874, -1, 18, -1, -1, -1, -1, -1, -1, 171, 276, -1, 153, 41, 130, 98, 734, 13, 1889, -1, -1, -1, 47, 90, 2521, -1, -1, 922, -1, 159, 61, 41, 79, 1105, 195, 42, 722, 12, -1, 51, -1, 874, -1, 598, 1373, -1, -1, 840, 75, 2466, -1, 96, 282, 1260, 32, -1, 52, 6, 82, 804, 101, -1, 153, 1830, -1, 43, -1, 209, 26, 193, 494, 1879, 261, 426, -1, 13860, 29, 210, 72, 621, 19754, -1, 683, 1250, 60, -1, 92, -1, 405, 208, 1304, 51, 444, -1, -1, 1680, 132, 298, 68, 13, -1, 168, 96, 31, 63, 97935, 528, 185, -1, 38, 86, 50, 223, 125, 148, 801, 338, 753, 54, 141, 1816, 53, -1, -1, 128, 160, 1830, 257, -1, -1, -1, 8519, 9, 282, -1, 9, 105, 101, -1, -1, -1, 1918, -1, -1, 692, 744, 115, 1083, 4000, 468, 3360, -1, 636, 22751, -1, -1, 273, 183, -1, 160, 278, -1, 236, 706, 121, 5, 1680, -1, 20, 209, 16, -1, 2254, -1, 38, 12, 108, 209, 20, 282, -1

In [341]:
#1f. and the number of characters in article’s paragraphs.
targetParagraphs_char_count = []
for x in range(0, len(raw_instance_train)):
    targetParagraphs_char_count.append(count_array_char(raw_instance_train[x]['targetParagraphs']))
print(targetParagraphs_char_count)

[3376, 364, 3421, 8954, 4810, 2990, 4901, 628, 271, 3624, 1975, 1799, 5747, 3968, 2788, 3616, 3763, 1450, 6292, 1263, 5117, 424, 424, 2954, 4520, 1010, 6392, 447, 67, 1747, 2743, 2502, 8611, -1, -1, 2670, 220, 1145, 5544, -1, 848, 2306, 2585, 42, 2892, 1151, 2926, 5406, 3395, 1276, 2236, 1910, 3767, 216, 2368, 1562, 1447, 4636, 2128, 3572, 810, 327, 1244, 1660, 3530, 866, 3561, 7478, -1, -1, 3073, 12485, 524, 1071, 2780, 2252, 1613, 2776, 2849, 2959, 9893, -1, 691, -1, 1018, 5943, 4166, -1, 143, 85, 2679, 1710, 2030, 4188, 843, 8640, 2373, 2947, 430, 5140, 2671, 3759, 12820, 829, 2341, 837, 436, -1, 2493, 1491, 2124, 5959, 2253, -1, 3381, 6929, 6959, 5751, 189, 634, 302, 5443, 2419, 2227, 4569, 41, 1187, 445, 614, 9544, 367, 4903, 5084, 1451, 2865, 493, 1491, 3862, 1176, 2197, 2911, 5460, 2037, 1456, 1822, 5767, 8111, 1542, 1751, 4413, 322, 3739, 401, 1028, 3426, 2361, 1343, 5747, 1617, -1, 3440, 361, 326, 3256, 3286, 3889, -1, 2426, 870, -1, 3391, 3083, 150, 3550, 924, 6649, 2627, 237

In [342]:
#2a. the number of words in post’s title.
postText_word_count = []
for x in range(0, len(raw_instance_train)):
    postText_word_count.append(count_tweet_array_word(raw_instance_train[x]['postText']))
print(postText_word_count)

[13, 15, 13, 11, 16, 12, 18, 11, 15, 14, 10, 15, 12, 14, 10, 21, 9, 12, 7, 15, 14, 8, 11, 8, 10, 15, 10, 11, 14, 13, 15, 5, 16, 18, 7, 11, 8, 19, 23, 20, 9, 15, 12, 16, 10, 13, 14, 10, 19, 12, 11, 14, 9, 9, 10, 11, 11, 14, 10, 18, 14, 8, 12, 12, 17, 15, 10, 15, 17, 16, 11, 5, 14, 10, 20, 9, 10, 9, 8, 15, 6, 10, 7, 22, 12, 16, 12, 17, 12, 11, 13, 9, 15, 10, 9, 12, 11, 11, 13, 10, 10, 13, 18, 12, 16, 9, 20, 16, 13, 12, 21, 11, 11, 10, 8, 15, 12, 16, 7, 15, 15, 9, 18, 11, 13, 12, 6, 13, 9, 12, 19, 12, 13, 13, 7, 15, 14, 21, 18, 11, 15, 14, 15, 11, 16, 15, 13, 13, 13, 12, 11, 12, 12, 13, 17, 13, 11, 12, 12, 15, 13, 9, 18, 11, 19, 19, 14, 13, 17, 23, 12, 8, 19, 15, 19, 10, 15, 6, 10, 17, 8, 18, 16, 15, 11, 20, 17, 12, 14, 8, 7, 9, 12, 17, 13, 16, 17, 12, 16, 7, 7, 12, 13, 10, 10, 11, 16, 17, 12, 13, 6, 17, 20, 13, 14, 11, 13, 9, 9, 13, 15, 14, 22, 10, 12, 17, 10, 13, 20, 15, 17, 9, 11, 11, 18, 17, 12, 9, 12, 11, 13, 15, 10, 10, 13, 15, 17, 10, 15, 15, 6, 11, 11, 7, 17, 16, 18, 10, 13, 7, 21

In [343]:
#2b. the number of words in article’s title.
targetTitle_word_count = []
for x in range(0, len(raw_instance_train)):
    targetTitle_word_count.append(count_tweet_string_word(raw_instance_train[x]['targetTitle']))
print(targetTitle_word_count)

[17, 8, 11, 11, 6, 12, 10, 11, 12, 14, 10, 12, 12, 23, 10, 14, 9, 24, 7, 10, 16, 10, 7, 5, 11, 14, 10, 11, 12, 7, 14, 8, 11, 10, 9, 11, 7, 19, 11, 1, 9, 13, 16, 3, 10, 300, 16, 10, 10, 296, 11, 21, 7, 9, 10, 11, 12, 9, 10, 11, 12, 8, 19, 10, 308, 296, 8, 4, 5, 9, 9, 4, 12, 12, 279, 9, 22, 9, 8, 9, 11, 8, 18, 9, 21, 8, 12, 15, 10, 1, 12, 9, 20, 10, 8, 13, 15, 11, 15, 6, 12, 12, 12, 11, 16, 10, 8, 11, 12, 10, 10, 11, 11, 11, 8, 10, 18, 7, 8, 14, 10, 8, 20, 15, 11, 12, 6, 14, 10, 12, 10, 9, 15, 17, 7, 10, 12, 6, 18, 17, 8, 9, 12, 13, 16, 17, 12, 13, 302, 7, 4, 11, 12, 10, 12, 18, 9, 12, 7, 5, 13, 4, 8, 11, 9, 9, 9, 13, 13, 2, 12, 8, 7, 11, 10, 10, 20, 12, 10, 10, 8, 10, 15, 25, 11, 36, 11, 12, 3, 8, 7, 9, 11, 18, 304, 11, 17, 12, 11, 9, 7, 10, 13, 5, 10, 10, 7, 20, 11, 12, 6, 9, 8, 8, 15, 11, 9, 10, 9, 302, 10, 10, 17, 8, 8, 12, 18, 12, 9, 4, 8, 8, 9, 21, 9, 13, 9, 10, 303, 12, 10, 15, 10, 12, 13, 26, 11, 9, 18, 7, 7, 13, 12, 14, 2, 14, 10, 7, 9, 19, 9, 13, 7, 13, 9, 10, 16, 11, 12, 12, 1

In [344]:
#2c. the number of words in article’s description. 
targetDescription_word_count = []
for x in range(0, len(raw_instance_train)):
    targetDescription_word_count.append(count_tweet_string_word(raw_instance_train[x]['targetDescription']))
print(targetDescription_word_count)

[31, 27, 24, 17, 22, -1, 27, 29, 28, 23, 24, 27, 5, 31, 14, 18, 21, 31, 22, 26, 14, 22, 6, 17, 27, -1, 28, -1, -1, 15, 23, 18, 20, 25, -1, 20, 26, 29, 53, 24, 21, 36, 22, -1, 43, 20, 20, 28, 25, 7, 32, 28, 18, 34, 26, 19, 20, 16, 22, 39, 21, 7, 23, 46, 28, 17, 17, 15, 20, 10, 39, 19, 23, 24, 14, 17, 30, 23, 23, 28, -1, 32, 28, 23, 23, 23, 20, 25, 31, -1, 13, 18, 34, 23, 10, 87, 23, 54, 41, 24, 34, 64, -1, 35, 14, 16, 44, 19, -1, 48, 27, 23, 24, 61, 20, 23, -1, 5, 40, -1, 25, 7, 21, 22, 14, -1, 8, -1, 36, -1, 27, 23, 33, 18, 23, 23, 33, 21, 25, 36, 22, 30, 31, 80, 24, 21, 13, 30, 14, 21, 16, 27, -1, 23, 27, 30, 40, 5, 28, -1, 46, 30, 28, 43, 27, 21, 29, 25, -1, -1, 21, 21, 37, 34, 10, 23, 26, 27, 22, 25, -1, 7, 15, 26, 30, 30, 36, 27, 11, 25, 25, 38, -1, 17, 18, 12, 37, 20, 19, 26, 11, 31, 34, 30, 32, 23, 23, 34, 32, 25, 14, 24, 32, 21, 25, 51, 32, 25, -1, 9, 39, 55, 9, 14, 10, -1, 22, 21, 18, 19, 34, -1, 22, 27, 37, 17, 33, 23, 15, 29, 23, 59, -1, 28, 24, 32, 45, 13, 17, 24, 18, 18, 23

In [345]:
#2d. the number of words in article’s keywords.
targetKeywords_word_count = []
for x in range(0, len(raw_instance_train)):
    targetKeywords_word_count.append(count_tweet_string_word(raw_instance_train[x]['targetKeywords']))
print(targetKeywords_word_count)

[13, 19, -1, 18, 11, -1, 10, -1, 20, -1, 7, 5, 15, 4, 6, 4, 9, 15, 8, -1, 7, -1, -1, 8, 31, -1, 25, -1, -1, -1, -1, -1, 10, 11, -1, 9, 26, 3, -1, -1, 10, 3, -1, -1, -1, 15, -1, 24, 11, 14, -1, 17, -1, 40, 2, -1, -1, 7, -1, 12, 8, 12, 9, -1, 35, 16, 19, 6, 6, 16, -1, 21, 7, -1, 10, 13, 17, 23, 14, 11, 13, -1, 13, 12, 6, -1, 18, 20, 1, -1, -1, -1, 11, 14, -1, 4, -1, -1, 50, 9, 3, -1, -1, -1, 6, -1, 22, 4, -1, -1, 8, 4, 7, -1, 11, 9, -1, -1, 6, -1, -1, 3, -1, -1, 9, -1, 7, -1, 3, -1, 18, 13, 10, 4, 18, 6, 5, -1, 4, 4, 10, 22, 12, -1, 16, 6, 10, -1, 15, 9, 4, 14, -1, 6, 32, 12, -1, 15, -1, -1, -1, 28, -1, -1, -1, -1, 21, 10, 13, -1, 12, 10, -1, -1, -1, 19, 9, -1, -1, 18, -1, 6, 23, 19, -1, 5, 10, 17, 4, 10, 21, -1, -1, 3, 11, 4, -1, 12, -1, -1, 25, 48, 11, 1, 14, 5, -1, 13, 28, 10, 8, -1, 14, -1, 20, -1, -1, 16, 27, 14, 11, -1, 8, 18, 8, -1, 13, 8, 10, -1, 8, -1, 12, 13, 18, 3, -1, 19, 17, 5, 9, -1, -1, 10, 8, 17, -1, 30, 3, 9, -1, 9, -1, 3, -1, 8, 9, 9, 6, -1, -1, 10, 13, 20, 40, -1, 10, 

In [346]:
#2e. the number of words in article’s captions.
targetCaptions_word_count = []
for x in range(0, len(raw_instance_train)):
    targetCaptions_word_count.append(count_tweet_array_word(raw_instance_train[x]['targetCaptions']))
print(targetCaptions_word_count)

[472, 248, 77, 191, 37, -1, -1, -1, 95, 23, 2, 1, -1, 4, 23, 22, 2, 333, 38, 9, -1, -1, -1, 12910, 37, -1, 47, 61, 80, -1, 18, 3, 454, 6063, 48, 180, -1, 1, -1, -1, -1, -1, -1, -1, 32, 50, -1, 30, 9, 28, 19, 136, 2, 250, -1, -1, -1, 13, 13, 523, -1, -1, 174, -1, 31, 13, 9, 15, 211, 34, 6, 153, 2, -1, 10, -1, 182, -1, 121, 267, -1, -1, 170, 12, 506, -1, 18, 60, 195, 6, -1, 10, 1, 15, 153, 4, -1, 31, 232, -1, 2, -1, 39, 7, 33, 96, 248, 46, 97, -1, 2576, 5, 35, 12, 124, 3835, -1, 118, 193, 14, -1, 27, -1, 85, 16, 305, 7, 90, -1, -1, 198, 22, 57, 3, 2, -1, 39, 20, 2, 5, 12285, 111, 40, -1, 6, 18, 6, 43, 26, 32, 179, 61, 152, 8, 29, 354, 9, -1, -1, 25, 23, 232, 48, -1, -1, -1, 1671, 2, 51, -1, 2, 12, 24, -1, -1, -1, 400, -1, -1, 137, 122, 22, 163, 798, 88, 688, -1, 128, 4255, -1, -1, 55, 42, -1, 32, 58, -1, 48, 136, 19, 1, 201, -1, 3, 43, 1, -1, 464, -1, 6, 2, 17, 46, 4, 60, -1, 5, -1, -1, 49, 41, 10, 149, 80, 180, 62, 620, 8, 18, 36, 21, 100, 4, 262, 205, -1, 41, 98, 88, -1, -1, 5, -1, 409

In [347]:
#2f. and the number of words in article’s paragraphs.
targetParagraphs_word_count = []
for x in range(0, len(raw_instance_train)):
    targetParagraphs_word_count.append(count_tweet_array_word(raw_instance_train[x]['targetParagraphs']))
print(targetParagraphs_word_count)

[721, 72, 703, 1897, 975, 720, 1015, 131, 53, 701, 379, 377, 1179, 780, 555, 728, 865, 286, 1215, 275, 1049, 90, 77, 634, 946, 240, 1307, 97, 16, 358, 563, 542, 1703, -1, -1, 547, 45, 244, 1281, -1, 189, 474, 520, 7, 558, 247, 583, 1107, 668, 276, 442, 345, 729, 45, 459, 302, 268, 1088, 437, 734, 160, 73, 242, 338, 769, 162, 761, 1526, -1, -1, 577, 2833, 99, 206, 565, 449, 335, 533, 630, 574, 2121, -1, 139, -1, 202, 1190, 886, -1, 31, 18, 536, 366, 431, 860, 157, 1890, 493, 618, 98, 1012, 544, 891, 2567, 157, 481, 172, 93, -1, 521, 342, 400, 1252, 438, -1, 642, 1344, 1614, 1152, 40, 110, 77, 1182, 442, 476, 879, 13, 252, 99, 118, 1871, 76, 948, 1110, 296, 615, 96, 313, 755, 256, 474, 592, 1173, 448, 309, 353, 1404, 1709, 353, 375, 840, 72, 796, 89, 194, 732, 463, 251, 1179, 315, -1, 703, 74, 72, 610, 699, 822, -1, 526, 192, -1, 664, 617, 37, 793, 215, 1384, 569, 457, 739, 1285, 82, 364, 129, 1633, 499, 1477, 131, 1767, -1, 355, 246, 1286, 2519, 102, 277, 525, 775, 972, 288, 401, 230, 6

In [348]:
#3a. the number of common words between article keywords and post’s title.
postText_word_frequency = []
for x in range(0, len(raw_instance_train)):
    keyList = keyWord_parse_(raw_instance_train[x]['targetKeywords'])
    postText_word_frequency.append(keyWordFrequency_array_(keyList, raw_instance_train[x]['postText']))
print(postText_word_frequency)

[0.6875, 0.047619047619047616, -1, 1.0909090909090908, 0.05555555555555555, -1, 0.0, -1, 0.05555555555555555, -1, 0.0, 1.8666666666666667, 0.14285714285714285, 2.0, 0.0, 0.03125, 0.1111111111111111, 0.5384615384615384, 0.0, -1, 0.0, -1, -1, 0.1, 0.18181818181818182, -1, 0.0, -1, -1, -1, -1, -1, 0.0, 0.0, -1, 0.14285714285714285, 0.0, 1.7272727272727273, -1, -1, 0.09090909090909091, 0.0, -1, -1, -1, 0.0, -1, 0.1, 0.0, 0.7857142857142857, -1, 0.35714285714285715, -1, 0.0, 0.0, -1, -1, 0.0, -1, 0.0, 0.05555555555555555, 0.125, 0.4166666666666667, -1, 0.10526315789473684, 0.058823529411764705, 0.0, 0.0, 0.0, 0.047619047619047616, -1, 0.0, 0.0, -1, 0.7391304347826086, 0.1111111111111111, 0.5, 0.0, 0.0, 0.0, 0.16666666666666666, -1, 0.7142857142857143, 0.1, 0.35714285714285715, -1, 0.15384615384615385, 0.0, 0.07692307692307693, -1, -1, -1, 0.0, 0.1, -1, 1.8461538461538463, -1, -1, 0.0, 0.06666666666666667, 2.0, -1, -1, -1, 0.0, -1, 0.05263157894736842, 0.047619047619047616, -1, -1, 0.0, 0.0,

In [349]:
#3b. the number of common words between article keywords and article description.
targetDescription_word_frequency = []
for x in range(0, len(raw_instance_train)):
    keyList = keyWord_parse_(raw_instance_train[x]['targetKeywords'])
    targetDescription_word_frequency.append(keyWordFrequency_string_(keyList, raw_instance_train[x]['targetDescription']))
print(targetDescription_word_frequency)

[0.18421052631578946, 0.0, -1, 0.5652173913043478, 0.08333333333333333, -1, 0.0, -1, 0.0625, -1, 0.041666666666666664, 1.8620689655172413, 0.125, 1.8529411764705883, 0.0, 0.0, 0.043478260869565216, 0.23529411764705882, 0.12, -1, 0.0, -1, -1, 0.05, 0.0, -1, 0.0, -1, -1, -1, -1, -1, 0.0, 0.0, -1, 0.0, 0.0, 2.0, -1, -1, 0.047619047619047616, 0.0, -1, -1, -1, 0.09523809523809523, -1, 0.034482758620689655, 0.0, 0.8571428571428571, -1, 0.1, -1, 0.08108108108108109, 0.0, -1, -1, 0.0, -1, 0.0, 0.045454545454545456, 0.0, 0.18518518518518517, -1, 0.034482758620689655, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, -1, 0.0, 0.0, -1, 0.9333333333333333, 0.0, 0.09375, 0.0, 0.15384615384615385, 0.0, -1, -1, 0.03125, 0.08, 0.18518518518518517, -1, 0.0, 0.1, 0.029411764705882353, -1, -1, -1, 0.0, 0.0, -1, 1.7765957446808511, -1, -1, 0.046511627906976744, 0.0, 1.894736842105263, -1, -1, -1, 0.0, -1, 0.0, 0.045454545454545456, -1, -1, 0.0, 0.0, 0.06060606060606061, -1, 0.13636363636363635, 0.0, -1, -1, 0.0238

In [350]:
#3c. the number of common words between article keywords and article captions.
targetCaptions_word_frequency = []
for x in range(0, len(raw_instance_train)):
    keyList = keyWord_parse_(raw_instance_train[x]['targetKeywords'])
    targetCaptions_word_frequency.append(keyWordFrequency_array_(keyList, raw_instance_train[x]['targetCaptions']))
print(targetCaptions_word_frequency)

[0.07865168539325842, 0.0, -1, 0.6759259259259259, 0.04878048780487805, -1, -1, -1, 0.0, -1, 0.0, 4.0, -1, 7.0, 0.0, 0.041666666666666664, 0.5, 0.17941952506596306, 0.0, -1, -1, -1, -1, 0.0006386715631486508, 0.045454545454545456, -1, 0.02040816326530612, -1, -1, -1, -1, -1, 0.011494252873563218, 0.0, -1, 0.009950248756218905, -1, 6.0, -1, -1, -1, -1, -1, -1, -1, 0.017241379310344827, -1, 0.0, 0.0, 0.6363636363636364, -1, 0.1375, -1, 0.006666666666666667, -1, -1, -1, 0.0, -1, 0.0, -1, -1, 0.10638297872340426, -1, 0.1111111111111111, 0.06666666666666667, 0.0, 0.0, 0.0, 0.0, -1, 0.011627906976744186, 0.0, -1, 0.6666666666666666, -1, 0.1958762886597938, -1, 0.0, 0.0, -1, -1, 0.11494252873563218, 0.07692307692307693, 0.022222222222222223, -1, 0.1, 0.0, 0.004484304932735426, -1, -1, -1, 0.0, 0.0625, -1, 8.5, -1, -1, 0.0, -1, 9.0, -1, -1, -1, 0.0, -1, 0.0, 0.04081632653061224, -1, -1, 0.0006626905235255136, 0.0, 0.04878048780487805, -1, 0.0072992700729927005, 0.0017090365306558428, -1, -1, 0

In [351]:
#3d. the number of common words between article keywords and article paragraphs.
targetParagraphs_word_frequency = []
for x in range(0, len(raw_instance_train)):
    keyList = keyWord_parse_(raw_instance_train[x]['targetKeywords'])
    targetParagraphs_word_frequency.append(keyWordFrequency_array_(keyList, raw_instance_train[x]['targetParagraphs']))
print(targetParagraphs_word_frequency)

[0.06463878326996197, 0.0375, -1, 0.7111319868482856, 0.009795191451469279, -1, 0.00608695652173913, -1, 0.03278688524590164, -1, 0.002364066193853428, 1.829683698296837, 0.03015075376884422, 1.774804905239688, 0.0, 0.012285012285012284, 0.005434782608695652, 0.10835913312693499, 0.02795698924731183, -1, 0.0, -1, -1, 0.0026560424966799467, 0.010880316518298714, -1, 0.0013698630136986301, -1, -1, -1, -1, -1, 0.014583333333333334, -1, -1, 0.027375201288244767, 0.0, 1.9083969465648856, -1, -1, 0.0196078431372549, 0.0035149384885764497, -1, -1, -1, 0.0149812734082397, -1, 0.016722408026755852, 0.0, 0.7547169811320755, -1, 0.08780487804878048, -1, 0.06382978723404255, 0.0, -1, -1, 0.0007980845969672786, -1, 0.0, 0.01, 0.011904761904761904, 0.07434944237918216, -1, 0.0187018701870187, 0.03296703296703297, 0.011737089201877934, 0.004753416518122401, -1, -1, -1, 0.0034733185980423114, 0.0, -1, 0.7774294670846394, 0.014227642276422764, 0.06426735218508997, 0.004878048780487805, 0.00816326530612

In [352]:
#4a. formal words ratio
formal_word_ratio = []
for x in range(0, len(raw_instance_train)):
    formal_word_ratio.append(count_formal_ratio(raw_instance_train[x]['postText']))
print(formal_word_ratio)

[0.8333333333333334, 0.7333333333333333, 0.8461538461538461, 0.9, 0.875, 0.7272727272727273, 0.7058823529411765, 0.875, 0.6923076923076923, 0.8461538461538461, 1.0, 0.6666666666666666, 0.8, 0.9166666666666666, 1.0, 0.6666666666666666, 0.5555555555555556, 0.75, 0.8333333333333334, 0.7142857142857143, 0.7142857142857143, 0.75, 0.8, 0.75, 0.8, 0.9090909090909091, 0.8888888888888888, 0.875, 0.5384615384615384, 0.8888888888888888, 0.8571428571428571, 0.8, 0.6, 0.3333333333333333, 0.4, 0.9, 0.75, 0.625, 0.7777777777777778, 0.7222222222222222, 0.8571428571428571, 0.6666666666666666, 0.8181818181818182, 0.6, 1.0, 0.9230769230769231, 0.8461538461538461, 1.0, 0.7142857142857143, 0.6666666666666666, 1.0, 0.9090909090909091, 0.5555555555555556, 0.6666666666666666, 1.0, 0.8888888888888888, 0.9090909090909091, 0.9, 0.75, 0.5625, 0.6923076923076923, 1.0, 0.8888888888888888, 0.7272727272727273, 0.7333333333333333, 0.8461538461538461, 0.6, 0.7272727272727273, 0.6666666666666666, 0.4666666666666667, 0.8

In [353]:
#5a. tags amount
tags_amount = []
for x in range(len(raw_instance_train)):
    tags_amount.append(tagFrequency_array(tags_parse("#,@"), paragraphs_tags_parse(raw_instance_train[x]['postText'])))
print(tags_amount)

[0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 4, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 1, 0, 1, 0, 3, 2, 1, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 1, 1, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 2, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2, 1, 3, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 3, 0, 0, 4, 0, 0, 2, 3, 0, 2, 2, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 4, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 2, 0, 0, 0, 0, 6, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 1, 0, 0, 

In [354]:
#Whether 'target Title' Start with 5W1H.
wh_start = []
for x in range(0, len(raw_instance_train)):
    wh_start.append(word_tokenize(raw_instance_train[x]['targetTitle'].lower())[0] in ['who', 'when', 'where', 'how', 'why','what'])
#Ethan's edit, I delete the [0] after raw_instance_train[x]['targetTitle']. It's a string already
wh_sample = lambda x: word_tokenize('What is it'.lower())[0] in ['who', 'when', 'where', 'how', 'why','what']
len(wh_start)

17535

In [355]:
#Foward reference: Whether 'target Title' has Demonstratives(this, that, those, these) 
hasDemonstrative = []
for y in range(0, len(raw_instance_train)):
    hasDemonstrative.append(sum(map(lambda x: str(x) in ['this', 'that', 'those','these'], 
                                     word_tokenize(raw_instance_train[y]['targetTitle'].lower()))) > 0)


In [356]:
#Foward reference: Whether 'target Title' has definite article('the')
hasDefinite = []
for y in range(0, len(raw_instance_train)):
    hasDefinite.append(sum(map(lambda x: str(x) == 'the', word_tokenize(raw_instance_train[y]
                                                                         ['targetTitle'].lower()))) > 0)


In [357]:
#Foward reference: Whether 'target Title' has 3rd person pronoun('he','she','his','her','him')
hasThirdPronoun = []
for y in range(0, len(raw_instance_train)):
    hasThirdPronoun.append(sum(map(lambda x: str(x) in ['he','she','his','her','him'], word_tokenize(raw_instance_train[y]['targetTitle'].lower()))) > 0)


In [358]:
#Foward reference: Whether 'target Title' Start with adverbs.
startsAdverb = []
for x in range(0, len(raw_instance_train)):
    startsAdverb.append(pos_tag(word_tokenize(raw_instance_train[x]['targetTitle']))[0][1] == 'ADV')
# print(pos_tag(word_tokenize(raw_instance_train[0]['targetTitle']))[0][1])
len(startsAdverb)

17535

In [359]:
countLongestWord = []

for x in range(0, len(raw_instance_train)):
    countLongestWord.append(len(max(raw_instance_train[x]['targetTitle'].split(), key=len)))
countLongestWord

[10,
 8,
 9,
 6,
 7,
 8,
 11,
 12,
 9,
 8,
 10,
 9,
 12,
 13,
 8,
 8,
 11,
 9,
 12,
 10,
 11,
 7,
 9,
 11,
 10,
 6,
 9,
 7,
 6,
 7,
 9,
 6,
 10,
 7,
 13,
 7,
 9,
 8,
 10,
 4,
 12,
 8,
 8,
 9,
 10,
 18,
 12,
 9,
 11,
 18,
 9,
 13,
 10,
 12,
 8,
 9,
 10,
 7,
 13,
 10,
 10,
 8,
 9,
 7,
 18,
 18,
 14,
 4,
 7,
 7,
 9,
 8,
 11,
 8,
 13,
 7,
 11,
 11,
 8,
 5,
 10,
 8,
 10,
 11,
 12,
 7,
 13,
 9,
 9,
 6,
 8,
 7,
 12,
 9,
 12,
 8,
 8,
 8,
 9,
 10,
 10,
 9,
 9,
 10,
 8,
 9,
 7,
 7,
 11,
 10,
 11,
 11,
 9,
 9,
 12,
 9,
 15,
 7,
 12,
 9,
 11,
 10,
 8,
 9,
 8,
 9,
 14,
 10,
 14,
 8,
 9,
 13,
 9,
 8,
 11,
 10,
 9,
 12,
 10,
 11,
 11,
 9,
 11,
 11,
 11,
 10,
 7,
 9,
 18,
 8,
 8,
 8,
 11,
 10,
 9,
 11,
 9,
 12,
 8,
 18,
 11,
 8,
 9,
 7,
 8,
 8,
 8,
 10,
 11,
 7,
 11,
 15,
 6,
 10,
 7,
 11,
 8,
 10,
 10,
 7,
 11,
 9,
 9,
 10,
 10,
 12,
 12,
 10,
 9,
 9,
 5,
 11,
 9,
 10,
 18,
 10,
 10,
 8,
 8,
 11,
 7,
 9,
 9,
 13,
 11,
 10,
 10,
 12,
 6,
 11,
 6,
 17,
 10,
 7,
 9,
 11,
 12,
 10,
 7,
 18,
 9,
 7,
 12,


In [360]:
averageLengthWords = []

for x in range(0, len(raw_instance_train)):
    averageLengthWords.append(sum(len(word) for word in raw_instance_train[x]['targetTitle'].split()) / len(raw_instance_train[x]['targetTitle'].split()))
averageLengthWords

[5.117647058823529,
 5.875,
 5.7,
 4.7272727272727275,
 4.833333333333333,
 4.909090909090909,
 6.7,
 5.090909090909091,
 4.0,
 4.642857142857143,
 6.2,
 5.166666666666667,
 5.5,
 5.695652173913044,
 5.7,
 4.071428571428571,
 6.555555555555555,
 4.916666666666667,
 6.0,
 4.7,
 5.333333333333333,
 4.8,
 5.625,
 5.8,
 5.2727272727272725,
 3.642857142857143,
 6.875,
 4.0,
 3.8,
 4.857142857142857,
 4.571428571428571,
 4.25,
 6.090909090909091,
 3.909090909090909,
 4.818181818181818,
 4.818181818181818,
 6.142857142857143,
 4.947368421052632,
 4.818181818181818,
 4.0,
 4.555555555555555,
 5.076923076923077,
 4.5,
 6.25,
 6.2,
 5.4366666666666665,
 5.1875,
 4.888888888888889,
 5.1,
 5.378378378378378,
 4.7272727272727275,
 6.181818181818182,
 6.857142857142857,
 7.0,
 4.4,
 5.2727272727272725,
 4.6,
 5.0,
 6.2,
 5.5,
 5.166666666666667,
 4.625,
 4.9,
 4.0,
 5.373376623376624,
 5.405405405405405,
 6.625,
 3.0,
 3.6666666666666665,
 4.555555555555555,
 5.666666666666667,
 5.25,
 6.63636363636

In [361]:
#Foward reference: Whether 'target Title' Start with Numbers.
startsPunctuation = []
for x in range(0, len(raw_instance_train)):
    startsPunctuation.append(raw_instance_train[0]['targetTitle'][0].isdigit())
startsPunctuation

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [362]:
pos_2gram_NNP_VBZ = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_VBZ.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'VBZ'))
pos_2gram_NNP_VBZ

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 8,
 0,
 1,
 0,
 8,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 7,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [363]:
pos_2gram_NNP_VBZ = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_VBZ.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'VBZ'))
pos_2gram_NNP_VBZ

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 8,
 0,
 1,
 0,
 8,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 7,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [364]:
pos_2gram_NN_IN = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NN_IN.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NN', 'IN'))
pos_2gram_NN_IN

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 12,
 2,
 0,
 0,
 12,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 12,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 7,
 1,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 14,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 13,
 0,
 1,
 2,
 0,
 1,
 0,
 2,
 0,
 0,


In [365]:
pos_2gram_PRP_VBP = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_PRP_VBP.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'PRP', 'VBP'))
pos_2gram_PRP_VBP

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,


In [366]:
title_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    title_LenCharacter.append(len(raw_instance_train[x]['targetTitle']))
title_LenCharacter

[103,
 54,
 66,
 62,
 34,
 64,
 76,
 67,
 64,
 79,
 71,
 73,
 77,
 153,
 66,
 70,
 67,
 141,
 48,
 56,
 75,
 57,
 52,
 33,
 68,
 64,
 63,
 54,
 47,
 40,
 77,
 41,
 77,
 53,
 63,
 63,
 49,
 112,
 63,
 4,
 49,
 78,
 65,
 29,
 72,
 1930,
 98,
 53,
 61,
 1887,
 63,
 157,
 54,
 56,
 53,
 68,
 55,
 53,
 71,
 64,
 73,
 44,
 118,
 49,
 1962,
 1895,
 61,
 16,
 27,
 49,
 60,
 25,
 83,
 57,
 1766,
 51,
 139,
 63,
 40,
 39,
 83,
 46,
 107,
 43,
 138,
 44,
 71,
 100,
 50,
 6,
 83,
 48,
 124,
 59,
 51,
 75,
 79,
 63,
 73,
 39,
 73,
 61,
 68,
 70,
 68,
 62,
 45,
 57,
 61,
 64,
 76,
 65,
 67,
 57,
 59,
 71,
 100,
 35,
 65,
 73,
 64,
 57,
 109,
 94,
 64,
 54,
 46,
 79,
 76,
 61,
 61,
 65,
 85,
 112,
 50,
 67,
 73,
 56,
 113,
 107,
 54,
 46,
 76,
 90,
 86,
 90,
 58,
 85,
 1921,
 39,
 25,
 68,
 72,
 70,
 79,
 106,
 52,
 77,
 42,
 39,
 80,
 32,
 51,
 62,
 64,
 56,
 51,
 77,
 77,
 13,
 70,
 64,
 37,
 68,
 45,
 73,
 101,
 49,
 62,
 50,
 45,
 71,
 77,
 151,
 67,
 201,
 74,
 71,
 23,
 60,
 30,
 61,
 51,
 103,

In [367]:
keyword_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    keyword_LenCharacter.append(len(raw_instance_train[x]['targetKeywords']))
keyword_LenCharacter

[78,
 129,
 0,
 132,
 70,
 0,
 66,
 0,
 161,
 0,
 47,
 43,
 108,
 21,
 86,
 19,
 55,
 94,
 58,
 0,
 45,
 0,
 0,
 70,
 242,
 0,
 191,
 0,
 0,
 0,
 0,
 0,
 122,
 60,
 0,
 66,
 194,
 32,
 0,
 0,
 71,
 29,
 0,
 0,
 0,
 109,
 0,
 161,
 92,
 108,
 0,
 132,
 0,
 330,
 13,
 0,
 0,
 77,
 0,
 88,
 61,
 76,
 63,
 0,
 249,
 117,
 117,
 43,
 34,
 107,
 0,
 165,
 83,
 0,
 88,
 96,
 114,
 156,
 94,
 61,
 107,
 0,
 88,
 86,
 50,
 0,
 142,
 139,
 4,
 0,
 0,
 0,
 86,
 102,
 0,
 26,
 0,
 0,
 363,
 80,
 23,
 0,
 0,
 0,
 43,
 0,
 158,
 29,
 0,
 0,
 69,
 31,
 73,
 0,
 76,
 58,
 0,
 0,
 43,
 0,
 0,
 30,
 0,
 0,
 102,
 0,
 64,
 0,
 18,
 0,
 152,
 108,
 81,
 21,
 118,
 43,
 32,
 0,
 27,
 21,
 79,
 138,
 103,
 0,
 116,
 50,
 103,
 0,
 102,
 59,
 29,
 106,
 0,
 76,
 238,
 77,
 0,
 108,
 0,
 0,
 0,
 217,
 0,
 0,
 0,
 0,
 147,
 60,
 93,
 0,
 81,
 112,
 0,
 0,
 0,
 128,
 52,
 0,
 0,
 121,
 0,
 45,
 133,
 127,
 0,
 31,
 70,
 122,
 30,
 70,
 116,
 0,
 0,
 17,
 86,
 45,
 0,
 93,
 0,
 0,
 160,
 350,
 78,
 6,
 103,
 43,

In [368]:
caption_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    caption_LenCharacter.append(len(raw_instance_train[x]['targetCaptions']))
caption_LenCharacter

[10,
 36,
 3,
 4,
 3,
 0,
 0,
 0,
 16,
 1,
 1,
 1,
 0,
 4,
 2,
 1,
 1,
 23,
 2,
 1,
 0,
 0,
 0,
 525,
 2,
 0,
 6,
 4,
 8,
 0,
 2,
 1,
 31,
 299,
 14,
 20,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 5,
 4,
 3,
 1,
 4,
 1,
 36,
 0,
 0,
 0,
 1,
 1,
 31,
 0,
 0,
 10,
 0,
 3,
 1,
 4,
 5,
 27,
 7,
 1,
 10,
 1,
 0,
 1,
 0,
 8,
 0,
 12,
 24,
 0,
 0,
 10,
 4,
 24,
 0,
 1,
 2,
 26,
 2,
 0,
 2,
 1,
 1,
 13,
 4,
 0,
 3,
 34,
 0,
 2,
 0,
 2,
 1,
 1,
 13,
 36,
 10,
 4,
 0,
 122,
 1,
 2,
 12,
 13,
 209,
 0,
 26,
 26,
 1,
 0,
 1,
 0,
 13,
 5,
 27,
 1,
 4,
 0,
 0,
 31,
 2,
 6,
 3,
 1,
 0,
 1,
 1,
 1,
 3,
 480,
 7,
 5,
 0,
 2,
 1,
 4,
 1,
 1,
 6,
 74,
 7,
 5,
 1,
 1,
 12,
 1,
 0,
 0,
 11,
 5,
 34,
 3,
 0,
 0,
 0,
 27,
 1,
 1,
 0,
 1,
 3,
 3,
 0,
 0,
 0,
 16,
 0,
 0,
 8,
 25,
 1,
 21,
 36,
 13,
 42,
 0,
 13,
 203,
 0,
 0,
 2,
 4,
 0,
 2,
 8,
 0,
 2,
 13,
 1,
 1,
 31,
 0,
 1,
 1,
 1,
 0,
 30,
 0,
 2,
 1,
 7,
 5,
 1,
 2,
 0,
 1,
 0,
 0,
 2,
 9,
 1,
 4,
 7,
 74,
 12,
 18,
 1,
 1,
 1,
 1,
 4,
 2,
 14,
 30,

In [369]:
description_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    description_LenCharacter.append(len(raw_instance_train[x]['targetDescription']))
description_LenCharacter

[184,
 166,
 153,
 114,
 125,
 0,
 154,
 171,
 140,
 157,
 134,
 160,
 36,
 187,
 76,
 100,
 128,
 192,
 129,
 133,
 81,
 127,
 43,
 91,
 147,
 0,
 173,
 0,
 0,
 106,
 146,
 107,
 138,
 161,
 0,
 125,
 152,
 173,
 301,
 156,
 122,
 183,
 140,
 0,
 257,
 105,
 123,
 172,
 138,
 37,
 195,
 180,
 117,
 189,
 166,
 131,
 127,
 88,
 112,
 231,
 175,
 34,
 148,
 302,
 150,
 85,
 125,
 90,
 111,
 60,
 248,
 130,
 146,
 175,
 84,
 101,
 175,
 156,
 135,
 151,
 0,
 204,
 184,
 147,
 158,
 127,
 112,
 163,
 173,
 0,
 112,
 104,
 179,
 134,
 59,
 481,
 133,
 309,
 208,
 153,
 222,
 301,
 0,
 209,
 90,
 96,
 255,
 111,
 0,
 301,
 155,
 136,
 152,
 359,
 147,
 143,
 0,
 29,
 225,
 0,
 132,
 36,
 132,
 152,
 79,
 0,
 51,
 0,
 232,
 0,
 186,
 156,
 191,
 113,
 123,
 124,
 194,
 134,
 145,
 225,
 152,
 151,
 164,
 442,
 156,
 115,
 85,
 167,
 78,
 130,
 89,
 161,
 0,
 138,
 155,
 211,
 247,
 36,
 198,
 0,
 269,
 155,
 143,
 261,
 148,
 124,
 170,
 146,
 0,
 0,
 125,
 121,
 186,
 212,
 56,
 129,
 170,


In [370]:
title_keyword_CharacterDiff = list((map(operator.sub, title_LenCharacter, keyword_LenCharacter)))
title_keyword_CharacterDiff

[25,
 -75,
 66,
 -70,
 -36,
 64,
 10,
 67,
 -97,
 79,
 24,
 30,
 -31,
 132,
 -20,
 51,
 12,
 47,
 -10,
 56,
 30,
 57,
 52,
 -37,
 -174,
 64,
 -128,
 54,
 47,
 40,
 77,
 41,
 -45,
 -7,
 63,
 -3,
 -145,
 80,
 63,
 4,
 -22,
 49,
 65,
 29,
 72,
 1821,
 98,
 -108,
 -31,
 1779,
 63,
 25,
 54,
 -274,
 40,
 68,
 55,
 -24,
 71,
 -24,
 12,
 -32,
 55,
 49,
 1713,
 1778,
 -56,
 -27,
 -7,
 -58,
 60,
 -140,
 0,
 57,
 1678,
 -45,
 25,
 -93,
 -54,
 -22,
 -24,
 46,
 19,
 -43,
 88,
 44,
 -71,
 -39,
 46,
 6,
 83,
 48,
 38,
 -43,
 51,
 49,
 79,
 63,
 -290,
 -41,
 50,
 61,
 68,
 70,
 25,
 62,
 -113,
 28,
 61,
 64,
 7,
 34,
 -6,
 57,
 -17,
 13,
 100,
 35,
 22,
 73,
 64,
 27,
 109,
 94,
 -38,
 54,
 -18,
 79,
 58,
 61,
 -91,
 -43,
 4,
 91,
 -68,
 24,
 41,
 56,
 86,
 86,
 -25,
 -92,
 -27,
 90,
 -30,
 40,
 -45,
 85,
 1819,
 -20,
 -4,
 -38,
 72,
 -6,
 -159,
 29,
 52,
 -31,
 42,
 39,
 80,
 -185,
 51,
 62,
 64,
 56,
 -96,
 17,
 -16,
 13,
 -11,
 -48,
 37,
 68,
 45,
 -55,
 49,
 49,
 62,
 -71,
 45,
 26,
 -56,
 24,
 6

In [371]:
title_caption_CharacterDiff = list((map(operator.sub, title_LenCharacter, caption_LenCharacter)))
title_caption_CharacterDiff

[93,
 18,
 63,
 58,
 31,
 64,
 76,
 67,
 48,
 78,
 70,
 72,
 77,
 149,
 64,
 69,
 66,
 118,
 46,
 55,
 75,
 57,
 52,
 -492,
 66,
 64,
 57,
 50,
 39,
 40,
 75,
 40,
 46,
 -246,
 49,
 43,
 49,
 111,
 63,
 4,
 49,
 78,
 65,
 29,
 71,
 1928,
 98,
 48,
 57,
 1884,
 62,
 153,
 53,
 20,
 53,
 68,
 55,
 52,
 70,
 33,
 73,
 44,
 108,
 49,
 1959,
 1894,
 57,
 11,
 0,
 42,
 59,
 15,
 82,
 57,
 1765,
 51,
 131,
 63,
 28,
 15,
 83,
 46,
 97,
 39,
 114,
 44,
 70,
 98,
 24,
 4,
 83,
 46,
 123,
 58,
 38,
 71,
 79,
 60,
 39,
 39,
 71,
 61,
 66,
 69,
 67,
 49,
 9,
 47,
 57,
 64,
 -46,
 64,
 65,
 45,
 46,
 -138,
 100,
 9,
 39,
 72,
 64,
 56,
 109,
 81,
 59,
 27,
 45,
 75,
 76,
 61,
 30,
 63,
 79,
 109,
 49,
 67,
 72,
 55,
 112,
 104,
 -426,
 39,
 71,
 90,
 84,
 89,
 54,
 84,
 1920,
 33,
 -49,
 61,
 67,
 69,
 78,
 94,
 51,
 77,
 42,
 28,
 75,
 -2,
 48,
 62,
 64,
 56,
 24,
 76,
 76,
 13,
 69,
 61,
 34,
 68,
 45,
 73,
 85,
 49,
 62,
 42,
 20,
 70,
 56,
 115,
 54,
 159,
 74,
 58,
 -180,
 60,
 30,
 59,
 47,
 

In [372]:
caption_title_CharacterRatio = list((map(operator.truediv, caption_LenCharacter, title_LenCharacter)))


In [373]:
description_title_CharacterRatio = list((map(operator.truediv, description_LenCharacter, title_LenCharacter)))


In [374]:
pos_3gram_NNP_NNP_VBZ = []
for x in range(0, len(raw_instance_train)):
    pos_3gram_NNP_NNP_VBZ.append(pos_3gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNP', 'VBZ'))
len(pos_3gram_NNP_NNP_VBZ)
print(pos_3gram_NNP_NNP_VBZ)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [375]:
pos_3gram_IN_NNP_NNP = []
for x in range(0, len(raw_instance_train)):
    pos_3gram_IN_NNP_NNP.append(pos_3gram_sum(raw_instance_train[x]['targetTitle'], 'IN','NNP', 'NNP'))
len(pos_3gram_IN_NNP_NNP)
print(pos_3gram_IN_NNP_NNP)

[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 1, 8, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 3, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 1, 1, 4, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 

In [376]:
pos_2gram_IN_NN = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_IN_NN.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'IN', 'NN'))
pos_2gram_IN_NN

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,


In [377]:
pos_2gram_NNP_NNS = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_NNS.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNS'))
pos_2gram_NNP_NNS

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 4,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [378]:
pos_2gram_IN_JJ = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_IN_JJ.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'IN', 'JJ'))
pos_2gram_IN_JJ

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 0,
 0,
 0,
 6,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,


In [379]:
import csv
from itertools import zip_longest

d = [pos_2gram_IN_JJ, pos_2gram_NNP_NNS, pos_2gram_IN_NN, pos_3gram_IN_NNP_NNP, pos_3gram_NNP_NNP_VBZ, 
     description_title_CharacterRatio, caption_title_CharacterRatio, title_caption_CharacterDiff, title_keyword_CharacterDiff, 
     description_LenCharacter, caption_LenCharacter, keyword_LenCharacter, title_LenCharacter, pos_2gram_PRP_VBP, pos_2gram_NN_IN, 
     pos_2gram_NNP_VBZ, startsPunctuation, averageLengthWords, countLongestWord, startsAdverb, hasThirdPronoun, 
     hasDemonstrative, wh_start, pos_3gram_NNP_NNP_NNP, pos_2gram_NNP_NNP, cal_pos_1grams_tag_list, 
     targetTitle_linsear_write_formula, targetTitle_gunning_fog, targetTitle_coleman_liau_index, 
     targetTitleCountWords, 
     postTitle_char_count, targetTitle_char_count, targetDescription_char_count, targetKeywords_char_count, 
     targetCaptions_char_count, targetParagraphs_char_count, postText_word_count, targetTitle_word_count,
     targetDescription_word_count, targetKeywords_word_count, targetCaptions_word_count, targetParagraphs_word_count,
     postText_word_frequency,targetDescription_word_frequency, targetCaptions_word_frequency, targetParagraphs_word_frequency,
     formal_word_ratio,tags_amount,label
    ]
export_data = zip_longest(*d, fillvalue = '')
with open('newFeatures_Ethan.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(("pos_2gram_IN_JJ","pos_2gram_NNP_NNS", "pos_2gram_IN_NN", "pos_3gram_IN_NNP_NNP", "pos_3gram_NNP_NNP_VBZ",
                   "description_title_CharacterRatio", "caption_title_CharacterRatio", "title_caption_CharacterDiff", 
                   "title_keyword_CharacterDiff", "description_LenCharacter", "caption_LenCharacter", 
                   "keyword_LenCharacter", "title_LenCharacter",  "pos_2gram_PRP_VBP", "pos_2gram_NN_IN", 
                   "pos_2gram_NNP_VBZ","startsPunctuation", "averageLengthWords", 
                   "countLongestWord", "startsAdverb", "hasThirdPronoun", "hasDemonstrative", "wh_start", 
                   "pos_3gram_NNP_NNP_NNP", "pos_2gram_NNP_NNP", "cal_pos_1grams_tag_list", 
                   "targetTitle_linsear_write_formula", "targetTitle_gunning_fog", "targetTitle_coleman_liau_index", 
                   "targetTitleCountWords",
                   "postTitle_char_count", "targetTitle_char_count","targetDescription_char_count", 
                   "targetKeywords_char_count", "targetCaptions_char_count", "targetParagraphs_char_count", 
                   "postText_word_count", "targetTitle_word_count", "targetDescription_word_count", 
                   "targetKeywords_word_count", "targetCaptions_word_count", "targetParagraphs_word_count",
                   "postText_word_frequency", "targetDescription_word_frequency", "targetCaptions_word_frequency", 
                   "targetParagraphs_word_frequency", "formal_word_ratio", "tags_amount","label"))
    wr.writerows(export_data)
myfile.close()

In [124]:
a = ["pos_2gram_IN_JJ","pos_2gram_NNP_NNS", "pos_2gram_IN_NN", "pos_3gram_IN_NNP_NNP", "pos_3gram_NNP_NNP_VBZ",
                   "description_title_CharacterRatio", "caption_title_CharacterRatio", "title_caption_CharacterDiff", 
                   "title_keyword_CharacterDiff", "description_LenCharacter", "caption_LenCharacter", 
                   "keyword_LenCharacter", "title_LenCharacter",  "pos_2gram_PRP_VBP", "pos_2gram_NN_IN", 
                   "pos_2gram_NNP_VBZ","startsPunctuation", "averageLengthWords", 
                   "countLongestWord", "startsAdverb", "hasThirdPronoun", "hasDemonstrative", "wh_start", 
                   "pos_3gram_NNP_NNP_NNP", "pos_2gram_NNP_NNP", "cal_pos_1grams_tag_list", 
                   "targetTitle_linsear_write_formula", "targetTitle_gunning_fog", "targetTitle_coleman_liau_index", 
                   "targetTitleCountWords",
                   "postTitle_char_count", "targetTitle_char_count","targetDescription_char_count", 
                   "targetKeywords_char_count", "targetCaptions_char_count", "targetParagraphs_char_count", 
                   "postText_word_count", "targetTitle_word_count", "targetDescription_word_count", 
                   "targetKeywords_word_count", "targetCaptions_word_count", "targetParagraphs_word_count",
                   "postText_word_frequency", "targetDescription_word_frequency", "targetCaptions_word_frequency", 
                   "targetParagraphs_word_frequency", "formal_word_ratio", "tags_amount","label"]

In [125]:
b = [0,0,0,0,0,1.7864077669902914,0.0970873786407767,93,25,184,10,78,103,0,0,1,False,5.117647058823529,10,False,False,False,False,0,0,1,6.5,20.564705882352943,12.12,17,63,87,154,66,2330,3376,13,17,31,13,472,721,0.6875,0.18421052631578946,0.07865168539325842,0.06463878326996197,0.8333333333333334,0,0]

In [127]:
features = [("word count", 'NUMERIC'),
            ("average word length", 'NUMERIC'),
            ("length of the longest word", 'NUMERIC'),
            ("whether start with number", ['True', 'False']),
            ("whether start with who/what/why/where/when/how", ['True', 'False']),
            ("label", ['0', '1'])]

49

In [126]:
###### dump data into arff file
import json # build-in library
import arff # https://pypi.python.org/pypi/liac-arff
data = {}
data.setdefault('attributes', features)
data.setdefault('description', '')
data.setdefault('relation', 'clickbait_sample')
data.setdefault('data', [])
for i in id_labels:
    tmp = [_ for _ in id_features[i]]
    tmp.append(str(id_labels[i]))
    data['data'].append(tmp)

with open('sample_train.arff', 'w') as f:
    f.write(arff.dumps(data))

49